In [155]:
#importing all libraries
import archook
import arcgis
import arcpy
from arcpy import env
from arcpy.sa import *
from arcpy import TableToTable_conversion as tt
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
arcpy.CheckOutExtension("Spatial")

#parameters: VAR identifies regions (used in resulting table)
#VAR = "RegionID"
VAR = "COUNTRY"
arcpy.env.overwriteOutput = True

#setting directories
dtb_path = r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\FAO SI Data (Tif Files)'
weights_path = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Crops GIS\crops_suitability\Global"
tables_path = r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Nikita indices'
regions_shape_path = r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Crops GIS\Regions_shp\Regions.shp'
csi_path = r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Caloric SI data'
countries_shape_path = r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Crops GIS\World_Countries.shp'
africa_shape_path = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\Downloads\afr_g2014_2013_0\afr_g2014_2013_0.shp"
hyde_path = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\HYDE Historical Data\HYDE Historical Data"
fao_prod_path = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\FAO Production Data"
trade_path = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Trade Access Data\Trade Access Data"
hmi_path = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Human Mobility Index\Human Mobility Index"

## The general part

In [2]:
# Functions to convert raster data to numpy arrays (and vice versa)

def raster_to_numpy(raster):
    descData = arcpy.Describe(raster)
    cellSize = descData.meanCellHeight
    extent = descData.Extent
    spatialReference = descData.spatialReference
    pnt = arcpy.Point(extent.XMin,extent.YMin)
    return arcpy.RasterToNumPyArray(raster, lower_left_corner=pnt, nodata_to_value=np.nan)

def numpy_to_raster(numpy_array):
    descData = arcpy.Describe(csi_path + '\\' + rasters_csi[0])
    cellSize = descData.meanCellHeight
    extent = descData.Extent
    spatialReference = descData.spatialReference
    pnt = arcpy.Point(extent.XMin,extent.YMin)
    newRaster = arcpy.NumPyArrayToRaster(numpy_array,pnt,cellSize,cellSize, value_to_nodata = np.nan)
    arcpy.DefineProjection_management(newRaster,spatialReference)
    return newRaster

def raster_to_numpy_0(raster):
    descData = arcpy.Describe(raster)
    extent = descData.Extent
    pnt = arcpy.Point(extent.XMin,extent.YMin)
    return arcpy.RasterToNumPyArray(raster, lower_left_corner = pnt, nodata_to_value = 0)

def sort_key(e):
    return e[1]

In [3]:
# Preparing CSI rasters

env.workspace = csi_path
rasters_raw = arcpy.ListRasters()
rasters_csi = []
for i in rasters_raw:
    if 'lo' in i: 
        rasters_csi.append(i)
print(f'We have CSI data on {len(rasters_csi)} crops.')


# Making np.arrays with CSI data for all crops

rasters_csi_np = []
for n in range(46):
    rasters_csi_np.append(raster_to_numpy(rasters_csi[n]))
    
rasters_csi_np_0 = []
for n in range(46):
    rasters_csi_np_0.append(raster_to_numpy_0(rasters_csi[n]))

We have CSI data on 46 crops.


In [4]:
# Preparing SI rasters

env.workspace = dtb_path
rasters_raw = arcpy.ListRasters()
rasters_si0 = []
for raster in rasters_raw:
    if 'sxLr0_' in raster:
        rasters_si0.append(raster.replace("sxLr0_", ""))
rasters_si = []
for raster in rasters_si0:
    rasters_si.append("sxLr0_" + raster)
    

    
rasters_si = sorted(rasters_si)
rasters_si[2], rasters_si[3] = rasters_si[3], rasters_si[2] 
rasters_si.insert(6, rasters_si[12])
del rasters_si[13]
rasters_si[15], rasters_si[16] = rasters_si[16], rasters_si[15] 
rasters_si.insert(17, rasters_si[28])
rasters_si[19], rasters_si[20] = rasters_si[20], rasters_si[19]
del rasters_si[29]
rasters_si[26], rasters_si[27] = rasters_si[27], rasters_si[26] 
rasters_si[26], rasters_si[28] = rasters_si[28], rasters_si[26] 
rasters_si[29], rasters_si[30] = rasters_si[30], rasters_si[29]
rasters_si[29], rasters_si[31] = rasters_si[31], rasters_si[29]
rasters_si[33], rasters_si[36] = rasters_si[36], rasters_si[33] 
rasters_si[35], rasters_si[37] = rasters_si[37], rasters_si[35] 
rasters_si[36], rasters_si[38] = rasters_si[38], rasters_si[36] 
rasters_si[37], rasters_si[38] = rasters_si[38], rasters_si[37] 

print(f'We have SI data on {len(rasters_si)} crops.')



rasters_si_np = []
for n in range(46):
    rasters_si_np.append(raster_to_numpy(Float(rasters_si[n])))

We have SI data on 46 crops.


In [5]:
# Checking the matching of CSI and SI raster files

dict(zip(rasters_csi, rasters_si))

{'alfalfalo.tif': 'sxLr0_alf_Alfalfa.tif',
 'bananalo.tif': 'sxLr0_ban_Banana.tif',
 'barleylo.tif': 'sxLr0_brl_Barley.tif',
 'buckwheatlo.tif': 'sxLr0_bck_Buckwheat.tif',
 'cabbagelo.tif': 'sxLr0_cab_Cabbage.tif',
 'carrotlo.tif': 'sxLr0_car_Carrot.tif',
 'cassavalo.tif': 'sxLr0_csv_Cassava.tif',
 'chickpealo.tif': 'sxLr0_chk_Chickpea.tif',
 'citruslo.tif': 'sxLr0_cit_Citrus.tif',
 'coconutlo.tif': 'sxLr0_coc_Coconut.tif',
 'coffeelo.tif': 'sxLr0_cof_Coffee.tif',
 'cottonlo.tif': 'sxLr0_cot_Cotton.tif',
 'cowpealo.tif': 'sxLr0_cow_Cowpea.tif',
 'flaxlo.tif': 'sxLr0_flx_Flax.tif',
 'foxtailmilletlo.tif': 'sxLr0_fml_FoxtailMillet.tif',
 'gramlo.tif': 'sxLr0_grm_Gram.tif',
 'groundnutlo.tif': 'sxLr0_grd_Groundnut.tif',
 'indricelo.tif': 'sxLr0_rcd_DrylandRice.tif',
 'jatrophalo.tif': 'sxLr0_jtr_Jatropha.tif',
 'maizelo.tif': 'sxLr0_mze_Maize.tif',
 'miscanthuslo.tif': 'sxLr0_mis_Miscanthus.tif',
 'oatlo.tif': 'sxLr0_oat_Oat.tif',
 'oilpalmlo.tif': 'sxLr0_olp_OilPalm.tif',
 'olivelo.tif':

In [6]:
crops_dict = {'alfalfalo.tif': 'alf','bananalo.tif': 'ban','barleylo.tif': 'brl','buckwheatlo.tif': 'bck','cabbagelo.tif': 'cab','carrotlo.tif': 'car','cassavalo.tif': 'csv','chickpealo.tif': 'chk','citruslo.tif': 'cit',
 'coconutlo.tif': 'cocn',
 'coffeelo.tif': 'cof',
 'cottonlo.tif': 'cot',
 'cowpealo.tif': 'cow',
 'flaxlo.tif': 'flx',
 'foxtailmilletlo.tif': 'fml',
 'gramlo.tif': 'grm',
 'groundnutlo.tif': 'grd',
 'indricelo.tif': 'rcd',
 'jatrophalo.tif': 'jtr',
 'maizelo.tif': 'mze',
 'miscanthuslo.tif': 'mis',
 'oatlo.tif': 'oat',
 'oilpalmlo.tif': 'olp',
 'olivelo.tif': 'olv',
 'onionlo.tif': 'oni',
 'pealo.tif': 'pea',
 'pearlmilletlo.tif': 'pml',
 'phaseolusbeanlo.tif': 'phb',
 'pigeonpealo.tif': 'pig',
 'rapeseedlo.tif': 'rsd',
 'reedcanarygrasslo.tif': 'rcg',
 'ricewetlo.tif': 'rcw',
 'ryelo.tif': 'rye',
 'sorghumlo.tif': 'srg',
 'soybeanlo.tif': 'soy',
 'sugarbeetlo.tif': 'sub',
 'sugarcanelo.tif': 'suc',
 'sunflowerlo.tif': 'sfl',
 'sweetpotatolo.tif': 'spo',
 'switchgrasslo.tif': 'swg',
 'tealo.tif': 'tea',
 'tobaccolo.tif': 'tob',
 'tomatolo.tif': 'tom',
 'wheatlo.tif': 'whe',
 'whitepotatolo.tif': 'wpo',
 'yamlo.tif': 'yam'}

crops_dict = dict(zip(crops_dict.values(), crops_dict.keys()))

In [7]:
# Preparing HYDE rasters

env.workspace = hyde_path
rasters_raw = arcpy.ListRasters()
rasters_hyde = []
for i in rasters_raw:
    if "AD.asc.tif" in i: 
        rasters_hyde.append(i)
print(f'We have HYDE data on {len(rasters_hyde)} files.')

for raster in rasters_hyde:
    descData = arcpy.Describe(dtb_path + '\\' + rasters_si[0])
    spatialReference = descData.spatialReference
    arcpy.DefineProjection_management(raster,spatialReference)

We have HYDE data on 16 files.


In [8]:
# Preparing FAO Production rasters

env.workspace = fao_prod_path
rasters_raw = arcpy.ListRasters()
rasters_fao_prod = []
for i in rasters_raw:
    if "2000" in i: 
        rasters_fao_prod.append(i)
print(f'We have FAO Production data on {len(rasters_fao_prod)} crops.')

We have FAO Production data on 27 crops.


In [9]:
# Preparing SI and CSI rasters for PRE1500

regions_pre1500 = pd.read_excel(tables_path + '\\FAO_CropAvailability_Pre1500.xlsx', sheet_name='Regions')
crops_pre1500 = pd.read_excel(tables_path + '\\FAO_CropAvailability_Pre1500.xlsx', sheet_name='Crops')
crops_pre1500 = crops_pre1500.iloc[:, 0:12]
regions_pre1500 = regions_pre1500.iloc[:, 0:8]

saf_crops = list(crops_pre1500[crops_pre1500['Subsaharan Africa']==1]['code'])
naf_crops = list(crops_pre1500[crops_pre1500['North Africa']==1]['code'])
eu_crops = list(crops_pre1500[crops_pre1500['Europe']==1]['code'])
asia_crops = list(crops_pre1500[crops_pre1500['Asia']==1]['code'])
oceania_crops = list(crops_pre1500[crops_pre1500['Oceania']==1]['code'])
america_crops = list(crops_pre1500[crops_pre1500['America']==1]['code'])

saf_rasters_csi = []
naf_rasters_csi = []
eu_rasters_csi = []
asia_rasters_csi = []
oceania_rasters_csi = []
america_rasters_csi = []

for crop in saf_crops:
    saf_rasters_csi.append(crops_dict[crop])
for crop in naf_crops:
    naf_rasters_csi.append(crops_dict[crop])
for crop in eu_crops:
    eu_rasters_csi.append(crops_dict[crop])
for crop in asia_crops:
    asia_rasters_csi.append(crops_dict[crop])
for crop in oceania_crops:
    oceania_rasters_csi.append(crops_dict[crop])
for crop in america_crops:
    america_rasters_csi.append(crops_dict[crop])

dict_names = dict(zip(rasters_csi, rasters_si))

saf_rasters_si = []
naf_rasters_si = []
eu_rasters_si = []
asia_rasters_si = []
oceania_rasters_si = []
america_rasters_si = []

for crop in saf_rasters_csi:
    saf_rasters_si.append(dict_names[crop])
for crop in naf_rasters_csi:
    naf_rasters_si.append(dict_names[crop])
for crop in eu_rasters_csi:
    eu_rasters_si.append(dict_names[crop])
for crop in asia_rasters_csi:
    asia_rasters_si.append(dict_names[crop])
for crop in oceania_rasters_csi:
    oceania_rasters_si.append(dict_names[crop])
for crop in america_rasters_csi:
    america_rasters_si.append(dict_names[crop])

    
fc = regions_shape_path
fields = ['RegionID', 'COUNTRY']
res = []
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        res.append(row)
ccodes = pd.DataFrame(res, columns = ['REGIONID', "COUNTRY"])
ccodes.loc[ccodes["COUNTRY"] == "Côte d'Ivoire", "COUNTRY"] = "C_te d'Ivoire"
ccodes.loc[ccodes["COUNTRY"] == "São Tomé and Príncipe", "COUNTRY"] = "S_o Tom_ and Pr_ncipe"
regions_pre1500 = regions_pre1500.merge(ccodes, left_on='rcode', right_on='REGIONID', how='left')

In [10]:
# Creating indicator-rasters for all crops which show whether SI in the specific cell is higher than 0.1 cutoff or not

env.workspace = dtb_path

rasters_si_ind_np = []
for i in tqdm(range(46)):
    rasters_si_ind_np.append((raster_to_numpy(Float(rasters_si[i]))>=1000)*1)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:18<00:00,  2.49it/s]


# DevEc

In [ ]:
env.workspace = dtb_path

out = Float(arcpy.sa.CellStatistics(rasters_si, 'MEAN', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(africa_shape_path, "ADM0_NAME", out,
                        r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\Downloads\RESULTS", "DATA", "ALL")
tt(r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\Downloads\RESULTS", r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\Downloads", 'Africa_si.csv')

In [ ]:
env.workspace = csi_path

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'MEAN', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(africa_shape_path, "ADM0_NAME", out,
                        r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\Downloads\RESULTS", "DATA", "ALL")
tt(r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\Downloads\RESULTS", r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\Downloads", 'Africa_csi.csv')

# New indices 

## A1_1

In [ ]:
env.workspace = csi_path

# Defining maximum CSI in each cell
csi_max_among_all_cell = Float(arcpy.sa.CellStatistics(rasters_csi, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum CSI = 0
csi_max_among_all_cells = SetNull(csi_max_among_all_cell, csi_max_among_all_cell, "VALUE <= 0")
csi_max_among_all_cells.save(tables_path + '\\A1\\A1_1\\csi_max')

In [ ]:
# Let's make an indicator for each cell which shows wheather a concrete crop's CSI equals to maximum CSI 

for raster in tqdm(rasters_csi):
    out = EqualTo(raster, csi_max_among_all_cells)
    Float(out).save(tables_path + '\\A1\\A1_1\\csi_max_ind\\' + raster)

In [ ]:
# Making np.arrays with indicators of max CSI for all crops 

env.workspace = tables_path + '\\A1\\A1_1\\csi_max_ind'

all_crops_np = []
for i in range(46):
    all_crops_np.append(raster_to_numpy(rasters_csi[i]))

In [ ]:
# Checking if there are few cells of different crops which equal to maximum CSI

overlay = np.zeros((2160, 4320)) 
for i in range(46):
    overlay = overlay + all_crops_np[i]

np.unique(overlay)

In [ ]:
# Fixing indicators of representing maximum CSI 

double_overlay = np.argwhere(overlay==2)
for i in range(46):
    for j in range(double_overlay.shape[0]):
        if all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] == 1:
            all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] = 0.5

In [ ]:
# Just checking the correctness of the received result

final = np.zeros((2160, 4320)) 
for i in range(46):
    final = final + all_crops_np[i]
    
print(np.argwhere(final>1))

# The list must be empty

In [ ]:
# Saving the final version of indicators of representing maximum CSI for all crops

for i in tqdm(range(46)):
    out = numpy_to_raster(all_crops_np[i])
    out.save(tables_path + '\\A1\\A1_1\\csi_max_ind\\' + rasters_csi[i])

In [12]:
# Calculating the share of cells for which crop c is optimal across regions (for all crops)
# Takes 30 min to compute

env.workspace = tables_path + '\\A1\\A1_1\\csi_max_ind'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A1\\A1_1\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [25:46<00:00, 33.62s/it]


In [13]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A1\\A1_1\\share'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A1\\A1_1\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:29<00:00,  1.54it/s]


In [14]:
env.workspace = tables_path + '\\A1\\A1_1\\share_squared'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A1\\A1_1\\RESULTS", "DATA", "MINIMUM")
tt(tables_path +"\\A1\\A1_1\\RESULTS", tables_path + '\\READY_country_level', 'A1_1_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A1_1_READY.csv'>

## A1_2

In [ ]:
# 'continent' is one word from set("saf", "naf", "eu", "asia", "oceania", "america") 

def computing_A1_2(rasters_csi, continent):

    env.workspace = csi_path

    # Defining maximum CSI in each cell
    csi_max_among_all_cell = Float(arcpy.sa.CellStatistics(rasters_csi, 'MAXIMUM', 'DATA'))

    # Avoiding cells where maximum CSI = 0
    csi_max_among_all_cells = SetNull(csi_max_among_all_cell, csi_max_among_all_cell, "VALUE <= 0")
    csi_max_among_all_cells.save(tables_path + '\\A1\\A1_2\\' + continent + '\\csi_max')

    # Let's make an indicator for each cell which shows wheather a concrete crop's CSI equals to maximum CSI 

    for raster in tqdm(rasters_csi):
        out = EqualTo(raster, csi_max_among_all_cells)
        Float(out).save(tables_path + '\\A1\\A1_2\\' + continent +  '\\csi_max_ind\\' + raster)
        
    # Making np.arrays with indicators of max CSI for all crops 
    env.workspace = tables_path + '\\A1\\A1_2\\' + continent +  '\\csi_max_ind'

    all_crops_np = []
    for i in range(len(rasters_csi)):
        all_crops_np.append(raster_to_numpy(rasters_csi[i]))
        
    # Checking if there are few cells of different crops which equal to maximum CSI

    overlay = np.zeros((2160, 4320)) 
    for i in range(len(rasters_csi)):
        overlay = overlay + all_crops_np[i]
        
    # Fixing indicators of representing maximum CSI 

    double_overlay = np.argwhere(overlay==2)
    for i in range(len(rasters_csi)):
        for j in range(double_overlay.shape[0]):
            if all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] == 1:
                all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] = 0.5
                
    # Saving the final version of indicators of representing maximum CSI for all crops

    for i in tqdm(range(len(rasters_csi))):
        out = numpy_to_raster(all_crops_np[i])
        out.save(tables_path + '\\A1\\A1_2\\' + continent + '\\csi_max_ind\\' + rasters_csi[i])
        
    # Calculating the share of cells for which crop c is optimal across regions (for all crops)
    
    env.workspace = tables_path + '\\A1\\A1_2\\' + continent + '\\csi_max_ind'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A1\\A1_2\\' + continent + '\\share\\' + raster)
        
    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A1\\A1_2\\' + continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A1\\A1_2\\' + continent + '\\share_squared\\' + raster)
        
    env.workspace = tables_path + '\\A1\\A1_2\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path + '\\A1\\A1_2\\' + continent + '\\RESULTS', "DATA", "MINIMUM")
    tt(tables_path + '\\A1\\A1_2\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A1_2_' + continent + '_READY.csv')

In [15]:
def computing_A1_2(rasters_csi, continent):

    env.workspace = tables_path + '\\A1\\A1_2\\' + continent + '\\csi_max_ind'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A1\\A1_2\\' + continent + '\\share\\' + raster)
        
    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A1\\A1_2\\' + continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A1\\A1_2\\' + continent + '\\share_squared\\' + raster)
        
    env.workspace = tables_path + '\\A1\\A1_2\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path + '\\A1\\A1_2\\' + continent + '\\RESULTS', "DATA", "MINIMUM")
    tt(tables_path + '\\A1\\A1_2\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A1_2_' + continent + '_READY.csv')

In [16]:
computing_A1_2(rasters_csi = saf_rasters_csi, continent = 'saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.65it/s]


In [17]:
computing_A1_2(rasters_csi = naf_rasters_csi, continent = 'naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:07<00:00,  2.51it/s]


In [18]:
computing_A1_2(rasters_csi = eu_rasters_csi, continent = 'eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:07<00:00,  2.37it/s]


In [19]:
computing_A1_2(rasters_csi = asia_rasters_csi, continent = 'asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:10<00:00,  2.51it/s]


In [20]:
computing_A1_2(rasters_csi = oceania_rasters_csi, continent = 'oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.08it/s]


In [21]:
computing_A1_2(rasters_csi = america_rasters_csi, continent = 'america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.57it/s]


In [22]:
# Then I manually renamed columns with computed indeces in resuting tables (from 'MIN' to 'continent')

In [11]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A1_2_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A1_2_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A1_2_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A1_2_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A1_2_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A1_2_america_READY.csv', sep = ';', decimal=',')

In [13]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A1_2'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A1_2']].to_excel(tables_path + '\\READY_country_level\\A1_2_READY.xlsx')

## A1_3

In [ ]:
env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")


In [ ]:
# Creating CSI rasters which satisfy both constraints:
# 1) max SI across all crops >= 0.1 (suitable cell for agriculture)
# 2) SI of the optimal crop >= 0.1 (suitable crop for growing)

for i in tqdm(range(46)):
    out = Con(dtb_path + '\\' + rasters_si[i], 1, 0, "VALUE >= 1000") * Raster(csi_path + '\\' + rasters_csi[i]) * Raster(cutoff_constraint)
    Float(out).save(tables_path + '\\A1\\A1_3\\rastersXind\\' + rasters_si[i])

In [ ]:
env.workspace = tables_path + '\\A1\\A1_3\\rastersXind'

# Defining maximum CSI in each cell
csi_max_among_all_cell = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum CSI = 0
csi_max_among_all_cells = SetNull(csi_max_among_all_cell, csi_max_among_all_cell, "VALUE <= 0")
csi_max_among_all_cells.save(tables_path + '\\A1\\A1_3\\csi_max')

In [ ]:
# Let's make an indicator for each cell which shows wheather a concrete crop's CSI equals to maximum CSI 

for raster in tqdm(rasters_si):
    out = EqualTo(raster, csi_max_among_all_cells)
    Float(out).save(tables_path + '\\A1\\A1_3\\csi_max_ind\\' + raster)

In [ ]:
# Making np.arrays with indicators of max CSI for all crops 

env.workspace = tables_path + '\\A1\\A1_3\\csi_max_ind'

all_crops_np = []
for i in range(46):
    all_crops_np.append(raster_to_numpy(rasters_si[i]))

In [ ]:
# Checking if there are few cells of different crops which equal to maximum CSI

overlay = np.zeros((2160, 4320)) 
for i in range(46):
    overlay = overlay + all_crops_np[i]

np.unique(overlay)

In [ ]:
# Fixing indicators of representing maximum CSI 

double_overlay = np.argwhere(overlay==2)
for i in range(46):
    for j in range(double_overlay.shape[0]):
        if all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] == 1:
            all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] = 0.5

In [ ]:
# Just checking the correctness of the received result

final = np.zeros((2160, 4320)) 
for i in range(46):
    final = final + all_crops_np[i]
    
print(np.argwhere(final>1))

# The list must be empty

In [ ]:
# Saving the final version of indicators of representing maximum CSI for all crops

env.workspace = tables_path + '\\A1\\A1_3\\csi_max_ind'

for i in tqdm(range(46)):
    out = numpy_to_raster(all_crops_np[i])
    out.save(tables_path + '\\A1\\A1_3\\csi_max_ind\\' + rasters_si[i])

In [11]:
# Calculating the share of cells for which crop c is optimal across regions (for all crops)
# Takes 30 min to compute

env.workspace = tables_path + '\\A1\\A1_3\\csi_max_ind'

for raster in tqdm(rasters_si):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A1\\A1_3\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [29:55<00:00, 39.02s/it]


In [12]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A1\\A1_3\\share'

for raster in tqdm(rasters_si):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A1\\A1_3\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:19<00:00,  2.35it/s]


In [13]:
env.workspace = tables_path + '\\A1\\A1_3\\share_squared'

out = Float(arcpy.sa.CellStatistics(rasters_si, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A1\\A1_3\\RESULTS", "DATA", "MINIMUM")
tt(tables_path +"\\A1\\A1_3\\RESULTS", tables_path + '\\READY_country_level', 'A1_3_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A1_3_READY.csv'>

## A1_4

In [ ]:
def computing_A1_4(rasters_csi, rasters_si, continent):
    
    env.workspace = dtb_path

    # Defining maximum SI in each cell
    max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

    # Avoiding cells where maximum SI <= 0.1
    cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")

    # Creating CSI rasters which satisfy both constraints:
    # 1) max SI across all crops >= 0.1 (suitable cell for agriculture)
    # 2) SI of the optimal crop >= 0.1 (suitable crop for growing)

    for i in tqdm(range(len(rasters_csi))):
        out = Con(dtb_path + '\\' + rasters_si[i], 1, 0, "VALUE >= 1000") * Raster(csi_path + '\\' + rasters_csi[i]) * Raster(cutoff_constraint)
        Float(out).save(tables_path + '\\A1\\A1_4\\' + continent + '\\rastersXind\\' + rasters_si[i])

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\rastersXind'
    
    # Defining maximum CSI in each cell
    csi_max_among_all_cell = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

    # Avoiding cells where maximum CSI = 0
    csi_max_among_all_cells = SetNull(csi_max_among_all_cell, csi_max_among_all_cell, "VALUE <= 0")
    csi_max_among_all_cells.save(tables_path + '\\A1\\A1_4\\' + continent + '\\csi_max')

    # Let's make an indicator for each cell which shows wheather a concrete crop's CSI equals to maximum CSI 

    for raster in tqdm(rasters_si):
        out = EqualTo(raster, csi_max_among_all_cells)
        Float(out).save(tables_path + '\\A1\\A1_4\\' + continent + '\\csi_max_ind\\' + raster)

    # Making np.arrays with indicators of max CSI for all crops 

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\csi_max_ind'

    all_crops_np = []
    for i in range(len(rasters_si)):
        all_crops_np.append(raster_to_numpy(rasters_si[i]))

    # Checking if there are few cells of different crops which equal to maximum CSI

    overlay = np.zeros((2160, 4320)) 
    for i in range(len(rasters_si)):
        overlay = overlay + all_crops_np[i]

    # Fixing indicators of representing maximum CSI 

    double_overlay = np.argwhere(overlay==2)
    for i in range(len(rasters_si)):
        for j in range(double_overlay.shape[0]):
            if all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] == 1:
                all_crops_np[i][double_overlay[j][0]][double_overlay[j][1]] = 0.5

    # Saving the final version of indicators of representing maximum CSI for all crops

    for i in tqdm(range(len(rasters_si))):
        out = numpy_to_raster(all_crops_np[i])
        out.save(tables_path + '\\A1\\A1_4\\' + continent + '\\csi_max_ind\\' + rasters_si[i])

    # Calculating the share of cells for which crop c is optimal across regions (for all crops)

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\csi_max_ind'

    for raster in tqdm(rasters_si):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A1\\A1_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\share'

    for raster in tqdm(rasters_si):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A1\\A1_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_si, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +"\\A1\\A1_4\\" + continent + "\\RESULTS", "DATA", "MINIMUM")
    tt(tables_path +"\\A1\\A1_4\\" + continent + "\\RESULTS", tables_path + '\\READY', 'A1_4_' + continent + '_READY.csv')

In [46]:
def computing_A1_4(rasters_csi, rasters_si, continent):

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\csi_max_ind'

    for raster in tqdm(rasters_si):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A1\\A1_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\share'

    for raster in tqdm(rasters_si):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A1\\A1_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A1\\A1_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_si, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +"\\A1\\A1_4\\" + continent + "\\RESULTS", "DATA", "MINIMUM")
    tt(tables_path +"\\A1\\A1_4\\" + continent + "\\RESULTS", tables_path + '\\READY_country_level', 'A1_4_' + continent + '_READY.csv')

In [47]:
computing_A1_4(rasters_csi = saf_rasters_csi, rasters_si = saf_rasters_si, continent = 'saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.50it/s]


In [48]:
computing_A1_4(rasters_csi = naf_rasters_csi, rasters_si = naf_rasters_si, continent = 'naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.73it/s]


In [49]:
computing_A1_4(rasters_csi = eu_rasters_csi, rasters_si = eu_rasters_si, continent = 'eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.70it/s]


In [50]:
computing_A1_4(rasters_csi = asia_rasters_csi, rasters_si = asia_rasters_si, continent = 'asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:10<00:00,  2.69it/s]


In [51]:
computing_A1_4(rasters_csi = america_rasters_csi, rasters_si = america_rasters_si, continent = 'america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  2.84it/s]


In [52]:
computing_A1_4(rasters_csi = oceania_rasters_csi, rasters_si = oceania_rasters_si, continent = 'oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.94it/s]


In [ ]:
# Then I manually renamed columns with computed indeces in resuting tables (from 'MIN' to 'continent')

In [34]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A1_4_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A1_4_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A1_4_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A1_4_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A1_4_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A1_4_america_READY.csv', sep = ';', decimal=',')

In [35]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MIN": lst_n[i]}, inplace=True)

In [36]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A1_4'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A1_4']].to_excel(tables_path + '\\READY_country_level\\A1_4_READY.xlsx')

## A2a_1

In [ ]:
# Finding out which crops make the TOP 3 in each cell
# Takes 15 min to compute

top3_crops_num = np.zeros((2160, 4320))
top3_crops_num = pd.DataFrame(top3_crops_num)
top3_crops_num = top3_crops_num.astype(object)

for i in tqdm(range(2160)):
    for j in range(4320):
        my_list = []
        for n in range(46):
            if rasters_csi_np_0[n][i][j] != 0:
                my_list.append((n, rasters_csi_np_0[n][i][j]))
        my_list = sorted(my_list, key = sort_key, reverse=True)
        if len(my_list)>=3:
            top3_crops_num[j][i] = [my_list[:3][k][0] for k in range(3)]

        
# In top3_crops_num, 0 means that there are no TOP 3 crops in the cell
# [a, c, b] means that crops by these ordinal numbers make the TOP 3 in the cell

In [ ]:
# Computing indicators of representing TOP 3 crops in each cell 

top3_ind = np.zeros((46, 2160, 4320))
for i in tqdm(range(2160)):
    for j in range(4320):
        if top3_crops_num[j][i] != 0:
            top3_ind[top3_crops_num[j][i][0]][i][j] = 1/3
            top3_ind[top3_crops_num[j][i][1]][i][j] = 1/3 
            top3_ind[top3_crops_num[j][i][2]][i][j] = 1/3 

In [ ]:
# Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

top3_crops_num_copy = top3_crops_num.copy()
top3_crops_num_copy = (top3_crops_num_copy == 0)*1
x = top3_crops_num_copy.astype('float64')
x = x.to_numpy()

for i in range(2160):
    for j in range(4320):
        if x[i][j]==1:
            x[i][j]=np.nan
        else: 
            x[i][j]=1

In [ ]:
# Adding 'No data' values to our indicators 

for i in range(46):
    top3_ind[i] = x * top3_ind[i]

# Saving indeces of representing TOP 3 crops in each cell 

env.workspace = csi_path

for i in tqdm(range(46)):
    out = numpy_to_raster(top3_ind[i])
    out.save(tables_path + '\\A2\\top3crops\\A2_1\\csi_top3_ind\\' + rasters_csi[i])

In [14]:
# Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)
# Takes 30 min to compute

env.workspace = tables_path + '\\A2\\top3crops\\A2_1\\csi_top3_ind'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A2\\top3crops\\A2_1\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [28:45<00:00, 37.50s/it]


In [15]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A2\\top3crops\\A2_1\\share'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A2\\top3crops\\A2_1\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:40<00:00,  1.13it/s]


In [16]:
env.workspace = tables_path + '\\A2\\top3crops\\A2_1\\share_squared'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A2\\top3crops\\A2_1\\RESULTS", "DATA", "MAXIMUM")
tt(tables_path +"\\A2\\top3crops\\A2_1\\RESULTS", tables_path + '\\READY_country_level', 'A2a_1_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A2a_1_READY.csv'>

## A2a_2

In [ ]:
def computing_A2a_2(rasters_csi, continent):

    env.workspace = csi_path

    rasters_csi_np_0 = []
    for n in range(len(rasters_csi)):
        rasters_csi_np_0.append(raster_to_numpy_0(rasters_csi[n]))

    # Finding out which crops make the TOP 3 in each cell

    top3_crops_num = np.zeros((2160, 4320))
    top3_crops_num = pd.DataFrame(top3_crops_num)
    top3_crops_num = top3_crops_num.astype(object)

    for i in tqdm(range(2160)):
        for j in range(4320):
            my_list = []
            for n in range(len(rasters_csi)):
                if rasters_csi_np_0[n][i][j] != 0:
                    my_list.append((n, rasters_csi_np_0[n][i][j]))
            my_list = sorted(my_list, key = sort_key, reverse=True)
            if len(my_list)>=3:
                top3_crops_num[j][i] = [my_list[:3][k][0] for k in range(3)]

    # Computing indicators of representing TOP 3 crops in each cell 

    top3_ind = np.zeros((len(rasters_csi), 2160, 4320))
    for i in tqdm(range(2160)):
        for j in range(4320):
            if top3_crops_num[j][i] != 0:
                top3_ind[top3_crops_num[j][i][0]][i][j] = 1/3
                top3_ind[top3_crops_num[j][i][1]][i][j] = 1/3 
                top3_ind[top3_crops_num[j][i][2]][i][j] = 1/3 


    # Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

    top3_crops_num_copy = top3_crops_num.copy()
    top3_crops_num_copy = (top3_crops_num_copy == 0)*1
    x = top3_crops_num_copy.astype('float64')
    x = x.to_numpy()

    for i in range(2160):
        for j in range(4320):
            if x[i][j]==1:
                x[i][j]=np.nan
            else: 
                x[i][j]=1

    # Adding 'No data' values to our indicators 

    for i in range(len(rasters_csi)):
        top3_ind[i] = x * top3_ind[i]

    # Saving indeces of representing TOP 3 crops in each cell 

    env.workspace = csi_path

    for i in tqdm(range(len(rasters_csi))):
        out = numpy_to_raster(top3_ind[i])
        out.save(tables_path + '\\A2\\top3crops\\A2_2\\' + continent + '\\csi_top3_ind\\' + rasters_csi[i])


    # Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

    env.workspace = tables_path + '\\A2\\top3crops\\A2_2\\' + continent + '\\csi_top3_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top3crops\\A2_2\\'+ continent + '\\share\\' + raster)


    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top3crops\\A2_2\\'+ continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top3crops\\A2_2\\' + continent + '\\share_squared\\' + raster)


    env.workspace = tables_path + '\\A2\\top3crops\\A2_2\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top3crops\\A2_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top3crops\\A2_2\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A2a_2_' + continent + '_READY.csv')

In [53]:
def computing_A2a_2(rasters_csi, continent):
    
    env.workspace = tables_path + '\\A2\\top3crops\\A2_2\\' + continent + '\\csi_top3_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top3crops\\A2_2\\'+ continent + '\\share\\' + raster)


    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top3crops\\A2_2\\'+ continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top3crops\\A2_2\\' + continent + '\\share_squared\\' + raster)


    env.workspace = tables_path + '\\A2\\top3crops\\A2_2\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top3crops\\A2_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top3crops\\A2_2\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A2a_2_' + continent + '_READY.csv')

In [54]:
computing_A2a_2(rasters_csi=saf_rasters_csi, continent='saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  2.90it/s]


In [55]:
computing_A2a_2(rasters_csi=naf_rasters_csi, continent='naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.69it/s]


In [56]:
computing_A2a_2(rasters_csi=asia_rasters_csi, continent='asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:12<00:00,  2.18it/s]


In [57]:
computing_A2a_2(rasters_csi=eu_rasters_csi, continent='eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.50it/s]


In [58]:
computing_A2a_2(rasters_csi=america_rasters_csi, continent='america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  3.15it/s]


In [59]:
computing_A2a_2(rasters_csi=oceania_rasters_csi, continent='oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.41it/s]


In [41]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A2a_2_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A2a_2_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A2a_2_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A2a_2_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A2a_2_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A2a_2_america_READY.csv', sep = ';', decimal=',')

In [42]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [43]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A2a_2'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A2a_2']].to_excel(tables_path + '\\READY_country_level\\A2a_2_READY.xlsx')

## A2a_3

In [ ]:
# Finding out which crops make the TOP 3 in each cell

top3_crops_num = np.zeros((2160, 4320))
top3_crops_num = pd.DataFrame(top3_crops_num)
top3_crops_num = top3_crops_num.astype(object)

for i in tqdm(range(2160)):
    for j in range(4320):
        my_list = []
        for n in range(46):
            if rasters_csi_np_0[n][i][j] != 0:
                my_list.append((n, rasters_csi_np_0[n][i][j]))
        my_list = sorted(my_list, key = sort_key, reverse=True)
        if len(my_list)>=3:
            inds = [my_list[:3][k][0] for k in range(3)]
            res = []
            for m in inds:
                if rasters_si_ind_np[m][i][j] == 1:
                    res.append(m)
            top3_crops_num[j][i] = res

        
# In top3_crops_num, 0 means that there are no TOP 3 crops in the cell
# [a, c, b] means that crops by these ordinal numbers make the TOP 3 in the cell

In [ ]:
# Computing indicators of representing TOP 3 crops in each cell 

top3_ind = np.zeros((46, 2160, 4320))
for i in tqdm(range(2160)):
    for j in range(4320):
        if top3_crops_num[j][i] != 0:
            if len(top3_crops_num[j][i])==3:
                top3_ind[top3_crops_num[j][i][0]][i][j] = 1/3
                top3_ind[top3_crops_num[j][i][1]][i][j] = 1/3 
                top3_ind[top3_crops_num[j][i][2]][i][j] = 1/3 
            elif len(top3_crops_num[j][i])==2:
                top3_ind[top3_crops_num[j][i][0]][i][j] = 1/2
                top3_ind[top3_crops_num[j][i][1]][i][j] = 1/2 
            elif len(top3_crops_num[j][i])==1:
                top3_ind[top3_crops_num[j][i][0]][i][j] = 1

In [ ]:
# Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

top3_crops_num_copy = top3_crops_num.copy()
top3_crops_num_copy = (top3_crops_num_copy == 0)*1
x = top3_crops_num_copy.astype('float64')
x = x.to_numpy()

for i in range(2160):
    for j in range(4320):
        if x[i][j]==1:
            x[i][j]=np.nan
        else: 
            x[i][j]=1

In [ ]:
env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))


In [ ]:
# Adding 'No data' values to our indicators 

for i in range(46):
    top3_ind[i] = x * top3_ind[i] * cutoff_constraint_np

# Saving indeces of representing TOP 3 crops in each cell 

env.workspace = csi_path

for i in tqdm(range(46)):
    out = numpy_to_raster(top3_ind[i])
    out.save(tables_path + '\\A2\\top3crops\\A2_3\\csi_top3_ind\\' + rasters_csi[i])

In [17]:
# Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

env.workspace = tables_path + '\\A2\\top3crops\\A2_3\\csi_top3_ind'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A2\\top3crops\\A2_3\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [26:25<00:00, 34.47s/it]


In [18]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A2\\top3crops\\A2_3\\share'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A2\\top3crops\\A2_3\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:20<00:00,  2.28it/s]


In [19]:
env.workspace = tables_path + '\\A2\\top3crops\\A2_3\\share_squared'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A2\\top3crops\\A2_3\\RESULTS", "DATA", "MAXIMUM")
tt(tables_path +"\\A2\\top3crops\\A2_3\\RESULTS", tables_path + '\\READY_country_level', 'A2a_3_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A2a_3_READY.csv'>

## A2a_4

In [ ]:
def computing_A2a_4(rasters_csi, rasters_si, continent):
    
    env.workspace = csi_path

    rasters_csi_np_0 = []
    for n in range(len(rasters_csi)):
        rasters_csi_np_0.append(raster_to_numpy_0(rasters_csi[n]))

    # Creating indicator-rasters for all crops which show whether SI in the specific cell is higher than 0.1 cutoff or not

    env.workspace = dtb_path

    rasters_si_ind_np = []
    for i in tqdm(range(len(rasters_csi))):
        rasters_si_ind_np.append((raster_to_numpy(Float(rasters_si[i]))>=1000)*1)

    # Finding out which crops make the TOP 3 in each cell

    top3_crops_num = np.zeros((2160, 4320))
    top3_crops_num = pd.DataFrame(top3_crops_num)
    top3_crops_num = top3_crops_num.astype(object)

    for i in tqdm(range(2160)):
        for j in range(4320):
            my_list = []
            for n in range(len(rasters_csi)):
                if rasters_csi_np_0[n][i][j] != 0:
                    my_list.append((n, rasters_csi_np_0[n][i][j]))
            my_list = sorted(my_list, key = sort_key, reverse=True)
            if len(my_list)>=3:
                inds = [my_list[:3][k][0] for k in range(3)]
                res = []
                for m in inds:
                    if rasters_si_ind_np[m][i][j] == 1:
                        res.append(m)
                top3_crops_num[j][i] = res

    # Computing indicators of representing TOP 3 crops in each cell 

    top3_ind = np.zeros((len(rasters_csi), 2160, 4320))
    for i in tqdm(range(2160)):
        for j in range(4320):
            if top3_crops_num[j][i] != 0:
                if len(top3_crops_num[j][i])==3:
                    top3_ind[top3_crops_num[j][i][0]][i][j] = 1/3
                    top3_ind[top3_crops_num[j][i][1]][i][j] = 1/3 
                    top3_ind[top3_crops_num[j][i][2]][i][j] = 1/3 
                elif len(top3_crops_num[j][i])==2:
                    top3_ind[top3_crops_num[j][i][0]][i][j] = 1/2
                    top3_ind[top3_crops_num[j][i][1]][i][j] = 1/2 
                elif len(top3_crops_num[j][i])==1:
                    top3_ind[top3_crops_num[j][i][0]][i][j] = 1

    # Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

    top3_crops_num_copy = top3_crops_num.copy()
    top3_crops_num_copy = (top3_crops_num_copy == 0)*1
    x = top3_crops_num_copy.astype('float64')
    x = x.to_numpy()

    for i in range(2160):
        for j in range(4320):
            if x[i][j]==1:
                x[i][j]=np.nan
            else: 
                x[i][j]=1

    env.workspace = dtb_path

    # Defining maximum SI in each cell
    max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

    # Avoiding cells where maximum SI <= 0.1
    cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
    cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))


    # Adding 'No data' values to our indicators 

    for i in range(len(rasters_csi)):
        top3_ind[i] = x * top3_ind[i] * cutoff_constraint_np

    # Saving indeces of representing TOP 3 crops in each cell 

    env.workspace = csi_path

    for i in tqdm(range(len(rasters_csi))):
        out = numpy_to_raster(top3_ind[i])
        out.save(tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\csi_top3_ind\\' + rasters_csi[i])

    # Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

    env.workspace = tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\csi_top3_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top3crops\\A2_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top3crops\\A2_4\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A2a_4_' + continent + '_READY.csv')

In [60]:
def computing_A2a_4(rasters_csi, rasters_si, continent):
    # Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

    env.workspace = tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\csi_top3_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A2\\top3crops\\A2_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top3crops\\A2_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top3crops\\A2_4\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A2a_4_' + continent + '_READY.csv')

In [61]:
computing_A2a_4(rasters_csi=saf_rasters_csi, rasters_si = saf_rasters_si, continent='saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  3.17it/s]


In [62]:
computing_A2a_4(rasters_csi=naf_rasters_csi, rasters_si = naf_rasters_si, continent='naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.88it/s]


In [63]:
computing_A2a_4(rasters_csi=asia_rasters_csi, rasters_si = asia_rasters_si, continent='asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:18<00:00,  1.46it/s]


In [64]:
computing_A2a_4(rasters_csi=eu_rasters_csi, rasters_si = eu_rasters_si, continent='eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  3.00it/s]


In [65]:
computing_A2a_4(rasters_csi=america_rasters_csi, rasters_si = america_rasters_si, continent='america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.71it/s]


In [66]:
computing_A2a_4(rasters_csi=oceania_rasters_csi, rasters_si = oceania_rasters_si, continent='oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.66it/s]


In [44]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A2a_4_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A2a_4_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A2a_4_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A2a_4_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A2a_4_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A2a_4_america_READY.csv', sep = ';', decimal=',')

In [45]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [46]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A2a_4'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A2a_4']].to_excel(tables_path + '\\READY_country_level\\A2a_4_READY.xlsx')

## A2b_1

In [ ]:
# Finding out which crops make the TOP 5 in each cell

top5_crops_num = np.zeros((2160, 4320))
top5_crops_num = pd.DataFrame(top5_crops_num)
top5_crops_num = top5_crops_num.astype(object)

for i in tqdm(range(2160)):
    for j in range(4320):
        my_list = []
        for n in range(46):
            if rasters_csi_np_0[n][i][j] != 0:
                my_list.append((n, rasters_csi_np_0[n][i][j]))
        my_list = sorted(my_list, key = sort_key, reverse=True)
        if len(my_list)>=5:
            top5_crops_num[j][i] = [my_list[:5][k][0] for k in range(5)]

In [ ]:
# Computing indicators of representing TOP 5 crops in each cell 

top5_ind = np.zeros((46, 2160, 4320))
for i in tqdm(range(2160)):
    for j in range(4320):
        if top5_crops_num[j][i] != 0:
            top5_ind[top5_crops_num[j][i][0]][i][j] = 1/5
            top5_ind[top5_crops_num[j][i][1]][i][j] = 1/5 
            top5_ind[top5_crops_num[j][i][2]][i][j] = 1/5 
            top5_ind[top5_crops_num[j][i][3]][i][j] = 1/5 
            top5_ind[top5_crops_num[j][i][4]][i][j] = 1/5 

In [ ]:
# Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

top5_crops_num_copy = top5_crops_num.copy()
top5_crops_num_copy = (top5_crops_num_copy == 0)*1
x = top5_crops_num_copy.astype('float64')
x = x.to_numpy()

for i in range(2160):
    for j in range(4320):
        if x[i][j]==1:
            x[i][j]=np.nan
        else: 
            x[i][j]=1

In [ ]:
# Adding 'No data' values to our indicators 

for i in range(46):
    top5_ind[i] = x * top5_ind[i]

# Saving indeces of representing TOP 5 crops in each cell 

env.workspace = csi_path

for i in tqdm(range(46)):
    out = numpy_to_raster(top5_ind[i])
    out.save(tables_path + '\\A2\\top5crops\\A2_1\\csi_top5_ind\\' + rasters_csi[i])

In [20]:
# Calculating the share of cells for which crop c is in TOP 5 across regions (for all crops)
# Takes 30 min to compute

env.workspace = tables_path + '\\A2\\top5crops\\A2_1\\csi_top5_ind'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A2\\top5crops\\A2_1\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [26:19<00:00, 34.35s/it]


In [21]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A2\\top5crops\\A2_1\\share'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A2\\top5crops\\A2_1\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:20<00:00,  2.19it/s]


In [22]:
env.workspace = tables_path + '\\A2\\top5crops\\A2_1\\share_squared'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A2\\top5crops\\A2_1\\RESULTS", "DATA", "MAXIMUM")
tt(tables_path +"\\A2\\top5crops\\A2_1\\RESULTS", tables_path + '\\READY_country_level', 'A2b_1_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A2b_1_READY.csv'>

## A2b_2

In [ ]:
def computing_A2b_2(rasters_csi, continent):

    env.workspace = csi_path

    rasters_csi_np_0 = []
    for n in range(len(rasters_csi)):
        rasters_csi_np_0.append(raster_to_numpy_0(rasters_csi[n]))

    # Finding out which crops make the TOP 5 in each cell

    top5_crops_num = np.zeros((2160, 4320))
    top5_crops_num = pd.DataFrame(top5_crops_num)
    top5_crops_num = top5_crops_num.astype(object)

    for i in tqdm(range(2160)):
        for j in range(4320):
            my_list = []
            for n in range(len(rasters_csi)):
                if rasters_csi_np_0[n][i][j] != 0:
                    my_list.append((n, rasters_csi_np_0[n][i][j]))
            my_list = sorted(my_list, key = sort_key, reverse=True)
            if len(my_list)>=5:
                top5_crops_num[j][i] = [my_list[:5][k][0] for k in range(5)]

    # Computing indicators of representing TOP 5 crops in each cell 

    top5_ind = np.zeros((len(rasters_csi), 2160, 4320))
    for i in tqdm(range(2160)):
        for j in range(4320):
            if top5_crops_num[j][i] != 0:
                top5_ind[top5_crops_num[j][i][0]][i][j] = 1/5
                top5_ind[top5_crops_num[j][i][1]][i][j] = 1/5 
                top5_ind[top5_crops_num[j][i][2]][i][j] = 1/5 
                top5_ind[top5_crops_num[j][i][3]][i][j] = 1/5 
                top5_ind[top5_crops_num[j][i][4]][i][j] = 1/5 


    # Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

    top5_crops_num_copy = top5_crops_num.copy()
    top5_crops_num_copy = (top5_crops_num_copy == 0)*1
    x = top5_crops_num_copy.astype('float64')
    x = x.to_numpy()

    for i in range(2160):
        for j in range(4320):
            if x[i][j]==1:
                x[i][j]=np.nan
            else: 
                x[i][j]=1

    # Adding 'No data' values to our indicators 

    for i in range(len(rasters_csi)):
        top5_ind[i] = x * top5_ind[i]

    # Saving indeces of representing TOP 5 crops in each cell 

    env.workspace = csi_path

    for i in tqdm(range(len(rasters_csi))):
        out = numpy_to_raster(top5_ind[i])
        out.save(tables_path + '\\A2\\top5crops\\A2_2\\' + continent + '\\csi_top5_ind\\' + rasters_csi[i])

    # Calculating the share of cells for which crop c is in TOP 5 across regions (for all crops)

    env.workspace = tables_path + '\\A2\\top5crops\\A2_2\\' + continent + '\\csi_top5_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top5crops\\A2_2\\'+ continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top5crops\\A2_2\\'+ continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top5crops\\A2_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A2\\top5crops\\A2_2\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top5crops\\A2_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top5crops\\A2_2\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A2b_2_' + continent + '_READY.csv')

In [67]:
def computing_A2b_2(rasters_csi, continent):
    env.workspace = tables_path + '\\A2\\top5crops\\A2_2\\' + continent + '\\csi_top5_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top5crops\\A2_2\\'+ continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top5crops\\A2_2\\'+ continent + '\\share'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top5crops\\A2_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A2\\top5crops\\A2_2\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top5crops\\A2_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top5crops\\A2_2\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A2b_2_' + continent + '_READY.csv')

In [68]:
computing_A2b_2(rasters_csi=saf_rasters_csi, continent='saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.39it/s]


In [69]:
computing_A2b_2(rasters_csi=naf_rasters_csi, continent='naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:07<00:00,  2.47it/s]


In [70]:
computing_A2b_2(rasters_csi=asia_rasters_csi, continent='asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:09<00:00,  2.82it/s]


In [71]:
computing_A2b_2(rasters_csi=eu_rasters_csi, continent='eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.96it/s]


In [72]:
computing_A2b_2(rasters_csi=america_rasters_csi, continent='america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.65it/s]


In [73]:
computing_A2b_2(rasters_csi=oceania_rasters_csi, continent='oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.21it/s]


In [47]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A2b_2_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A2b_2_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A2b_2_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A2b_2_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A2b_2_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A2b_2_america_READY.csv', sep = ';', decimal=',')

In [48]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [49]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A2b_2'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A2b_2']].to_excel(tables_path + '\\READY_country_level\\A2b_2_READY.xlsx')

## A2b_3

In [ ]:
# Finding out which crops make the TOP 5 in each cell

top5_crops_num = np.zeros((2160, 4320))
top5_crops_num = pd.DataFrame(top5_crops_num)
top5_crops_num = top5_crops_num.astype(object)

for i in tqdm(range(2160)):
    for j in range(4320):
        my_list = []
        for n in range(46):
            if rasters_csi_np_0[n][i][j] != 0:
                my_list.append((n, rasters_csi_np_0[n][i][j]))
        my_list = sorted(my_list, key = sort_key, reverse=True)
        if len(my_list)>=5:
            inds = [my_list[:5][k][0] for k in range(5)]
            res = []
            for m in inds:
                if rasters_si_ind_np[m][i][j] == 1:
                    res.append(m)
            top5_crops_num[j][i] = res

In [ ]:
# Computing indicators of representing TOP 5 crops in each cell 

top5_ind = np.zeros((46, 2160, 4320))
for i in tqdm(range(2160)):
    for j in range(4320):
        if top5_crops_num[j][i] != 0:
            if len(top5_crops_num[j][i])==5:
                top5_ind[top5_crops_num[j][i][0]][i][j] = 1/5
                top5_ind[top5_crops_num[j][i][1]][i][j] = 1/5 
                top5_ind[top5_crops_num[j][i][2]][i][j] = 1/5
                top5_ind[top5_crops_num[j][i][3]][i][j] = 1/5 
                top5_ind[top5_crops_num[j][i][4]][i][j] = 1/5 
            elif len(top5_crops_num[j][i])==4:
                top5_ind[top5_crops_num[j][i][0]][i][j] = 1/4
                top5_ind[top5_crops_num[j][i][1]][i][j] = 1/4 
                top5_ind[top5_crops_num[j][i][2]][i][j] = 1/4
                top5_ind[top5_crops_num[j][i][3]][i][j] = 1/4 
            elif len(top5_crops_num[j][i])==3:
                top5_ind[top5_crops_num[j][i][0]][i][j] = 1/3
                top5_ind[top5_crops_num[j][i][1]][i][j] = 1/3 
                top5_ind[top5_crops_num[j][i][2]][i][j] = 1/3 
            elif len(top5_crops_num[j][i])==2:
                top5_ind[top5_crops_num[j][i][0]][i][j] = 1/2
                top5_ind[top5_crops_num[j][i][1]][i][j] = 1/2 
            elif len(top5_crops_num[j][i])==1:
                top5_ind[top5_crops_num[j][i][0]][i][j] = 1

In [ ]:
# Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

top5_crops_num_copy = top5_crops_num.copy()
top5_crops_num_copy = (top5_crops_num_copy == 0)*1
x = top5_crops_num_copy.astype('float64')
x = x.to_numpy()

for i in range(2160):
    for j in range(4320):
        if x[i][j]==1:
            x[i][j]=np.nan
        else: 
            x[i][j]=1

In [ ]:
env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))

In [ ]:
# Adding 'No data' values to our indicators 

for i in range(46):
    top5_ind[i] = x * top5_ind[i] * cutoff_constraint_np

# Saving indeces of representing TOP 5 crops in each cell 

env.workspace = csi_path

for i in tqdm(range(46)):
    out = numpy_to_raster(top5_ind[i])
    out.save(tables_path + '\\A2\\top5crops\\A2_3\\csi_top5_ind\\' + rasters_csi[i])

In [23]:
# Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

env.workspace = tables_path + '\\A2\\top5crops\\A2_3\\csi_top5_ind'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A2\\top5crops\\A2_3\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [26:16<00:00, 34.27s/it]


In [24]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A2\\top5crops\\A2_3\\share'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A2\\top5crops\\A2_3\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:29<00:00,  1.57it/s]


In [25]:
env.workspace = tables_path + '\\A2\\top5crops\\A2_3\\share_squared'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A2\\top5crops\\A2_3\\RESULTS", "DATA", "MAXIMUM")
tt(tables_path +"\\A2\\top5crops\\A2_3\\RESULTS", tables_path + '\\READY_country_level', 'A2b_3_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A2b_3_READY.csv'>

## A2b_4

In [ ]:
def computing_A2b_4(rasters_csi, rasters_si, continent):
    
    env.workspace = csi_path

    rasters_csi_np_0 = []
    for n in range(len(rasters_csi)):
        rasters_csi_np_0.append(raster_to_numpy_0(rasters_csi[n]))

    # Creating indicator-rasters for all crops which show whether SI in the specific cell is higher than 0.1 cutoff or not

    env.workspace = dtb_path

    rasters_si_ind_np = []
    for i in tqdm(range(len(rasters_si))):
        rasters_si_ind_np.append((raster_to_numpy(Float(rasters_si[i]))>=1000)*1)

    # Finding out which crops make the TOP 5 in each cell

    top5_crops_num = np.zeros((2160, 4320))
    top5_crops_num = pd.DataFrame(top5_crops_num)
    top5_crops_num = top5_crops_num.astype(object)

    for i in tqdm(range(2160)):
        for j in range(4320):
            my_list = []
            for n in range(len(rasters_csi)):
                if rasters_csi_np_0[n][i][j] != 0:
                    my_list.append((n, rasters_csi_np_0[n][i][j]))
            my_list = sorted(my_list, key = sort_key, reverse=True)
            if len(my_list)>=5:
                inds = [my_list[:5][k][0] for k in range(5)]
                res = []
                for m in inds:
                    if rasters_si_ind_np[m][i][j] == 1:
                        res.append(m)
                top5_crops_num[j][i] = res

    # Computing indicators of representing TOP 5 crops in each cell 

    top5_ind = np.zeros((len(rasters_csi), 2160, 4320))
    for i in tqdm(range(2160)):
        for j in range(4320):
            if top5_crops_num[j][i] != 0:
                if len(top5_crops_num[j][i])==5:
                    top5_ind[top5_crops_num[j][i][0]][i][j] = 1/5
                    top5_ind[top5_crops_num[j][i][1]][i][j] = 1/5 
                    top5_ind[top5_crops_num[j][i][2]][i][j] = 1/5
                    top5_ind[top5_crops_num[j][i][3]][i][j] = 1/5 
                    top5_ind[top5_crops_num[j][i][4]][i][j] = 1/5 
                elif len(top5_crops_num[j][i])==4:
                    top5_ind[top5_crops_num[j][i][0]][i][j] = 1/4
                    top5_ind[top5_crops_num[j][i][1]][i][j] = 1/4 
                    top5_ind[top5_crops_num[j][i][2]][i][j] = 1/4
                    top5_ind[top5_crops_num[j][i][3]][i][j] = 1/4 
                elif len(top5_crops_num[j][i])==3:
                    top5_ind[top5_crops_num[j][i][0]][i][j] = 1/3
                    top5_ind[top5_crops_num[j][i][1]][i][j] = 1/3 
                    top5_ind[top5_crops_num[j][i][2]][i][j] = 1/3 
                elif len(top5_crops_num[j][i])==2:
                    top5_ind[top5_crops_num[j][i][0]][i][j] = 1/2
                    top5_ind[top5_crops_num[j][i][1]][i][j] = 1/2 
                elif len(top5_crops_num[j][i])==1:
                    top5_ind[top5_crops_num[j][i][0]][i][j] = 1

    # Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

    top5_crops_num_copy = top5_crops_num.copy()
    top5_crops_num_copy = (top5_crops_num_copy == 0)*1
    x = top5_crops_num_copy.astype('float64')
    x = x.to_numpy()

    for i in range(2160):
        for j in range(4320):
            if x[i][j]==1:
                x[i][j]=np.nan
            else: 
                x[i][j]=1


    env.workspace = dtb_path

    # Defining maximum SI in each cell
    max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

    # Avoiding cells where maximum SI <= 0.1
    cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
    cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))

    # Adding 'No data' values to our indicators 

    for i in range(len(rasters_csi)):
        top5_ind[i] = x * top5_ind[i] * cutoff_constraint_np

    # Saving indeces of representing TOP 5 crops in each cell 

    env.workspace = csi_path

    for i in tqdm(range(len(rasters_csi))):
        out = numpy_to_raster(top5_ind[i])
        out.save(tables_path + '\\A2\\top5crops\\A2_4\\'+continent+'\\csi_top5_ind\\' + rasters_csi[i])


    # Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

    env.workspace = tables_path + '\\A2\\top5crops\\A2_4\\'+continent+'\\csi_top5_ind'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share\\' + raster)


    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top5crops\\A2_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top5crops\\A2_4\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A2b_4_' + continent + '_READY.csv')

In [74]:
def computing_A2b_4(rasters_csi, rasters_si, continent):
    env.workspace = tables_path + '\\A2\\top5crops\\A2_4\\'+continent+'\\csi_top5_ind'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share\\' + raster)


    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A2\\top5crops\\A2_4\\' + continent + '\\share_squared'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A2\\top5crops\\A2_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A2\\top5crops\\A2_4\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A2b_4_' + continent + '_READY.csv')

In [75]:
computing_A2b_4(rasters_csi=saf_rasters_csi, rasters_si = saf_rasters_si, continent='saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.74it/s]


In [76]:
computing_A2b_4(rasters_csi=naf_rasters_csi, rasters_si = naf_rasters_si, continent='naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:09<00:00,  1.85it/s]


In [77]:
computing_A2b_4(rasters_csi=asia_rasters_csi, rasters_si = asia_rasters_si, continent='asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:09<00:00,  2.96it/s]


In [78]:
computing_A2b_4(rasters_csi=eu_rasters_csi, rasters_si = eu_rasters_si, continent='eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  3.05it/s]


In [79]:
computing_A2b_4(rasters_csi=america_rasters_csi, rasters_si = america_rasters_si, continent='america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.69it/s]


In [80]:
computing_A2b_4(rasters_csi=oceania_rasters_csi, rasters_si = oceania_rasters_si, continent='oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.55it/s]


In [50]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A2b_4_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A2b_4_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A2b_4_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A2b_4_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A2b_4_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A2b_4_america_READY.csv', sep = ';', decimal=',')

In [51]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [52]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A2b_4'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A2b_4']].to_excel(tables_path + '\\READY_country_level\\A2b_4_READY.xlsx')

## A3_1

In [ ]:
# Finding out which crops make the TOP 5 in each cell

top5_crops_num = np.zeros((2160, 4320))
top5_crops_num = pd.DataFrame(top5_crops_num)
top5_crops_num = top5_crops_num.astype(object)

for i in tqdm(range(2160)):
    for j in range(4320):
        my_list = []
        for n in range(46):
            if rasters_csi_np_0[n][i][j] != 0:
                my_list.append([n, rasters_csi_np_0[n][i][j]])
        my_list = sorted(my_list, key = sort_key, reverse=True)
        if len(my_list)>=5:
            top5_crops_num[j][i] = [(my_list[:5][k][0], my_list[:5][k][1]) for k in range(5)]

In [ ]:
# Computing indicators of representing TOP 5 crops in each cell 

top5_ind = np.zeros((46, 2160, 4320))
for i in tqdm(range(2160)):
    for j in range(4320):
        if top5_crops_num[j][i] != 0:
            sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] + top5_crops_num[j][i][3][1] + top5_crops_num[j][i][4][1]
            top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
            top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
            top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
            top5_ind[top5_crops_num[j][i][3][0]][i][j] = top5_crops_num[j][i][3][1]/sum_top5_csi
            top5_ind[top5_crops_num[j][i][4][0]][i][j] = top5_crops_num[j][i][4][1]/sum_top5_csi

In [ ]:
# Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

top5_crops_num_copy = top5_crops_num.copy()
top5_crops_num_copy = (top5_crops_num_copy == 0)*1
x = top5_crops_num_copy.astype('float64')
x = x.to_numpy()

for i in range(2160):
    for j in range(4320):
        if x[i][j]==1:
            x[i][j]=np.nan
        else: 
            x[i][j]=1

In [ ]:
# Adding 'No data' values to our indicators 

for i in range(46):
    top5_ind[i] = x * top5_ind[i]

# Saving indeces of representing TOP 5 crops in each cell 

env.workspace = csi_path

for i in tqdm(range(46)):
    out = numpy_to_raster(top5_ind[i])
    out.save(tables_path + '\\A3\\A3_1\\csi_top5_ind\\' + rasters_csi[i])

In [26]:
# Calculating the share of cells for which crop c is in TOP 5 across regions (for all crops)

env.workspace = tables_path + '\\A3\\A3_1\\csi_top5_ind'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A3\\A3_1\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [26:10<00:00, 34.15s/it]


In [27]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A3\\A3_1\\share'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A3\\A3_1\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:19<00:00,  2.37it/s]


In [28]:
env.workspace = tables_path + '\\A3\\A3_1\\share_squared'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A3\\A3_1\\RESULTS", "DATA", "MAXIMUM")
tt(tables_path +"\\A3\\A3_1\\RESULTS", tables_path + '\\READY_country_level', 'A3_1_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A3_1_READY.csv'>

## A3_2

In [ ]:
def computing_A3_2(rasters_csi, continent):

    env.workspace = csi_path

    rasters_csi_np_0 = []
    for n in range(len(rasters_csi)):
        rasters_csi_np_0.append(raster_to_numpy_0(rasters_csi[n]))

    # Finding out which crops make the TOP 5 in each cell

    top5_crops_num = np.zeros((2160, 4320))
    top5_crops_num = pd.DataFrame(top5_crops_num)
    top5_crops_num = top5_crops_num.astype(object)

    for i in tqdm(range(2160)):
        for j in range(4320):
            my_list = []
            for n in range(len(rasters_csi)):
                if rasters_csi_np_0[n][i][j] != 0:
                    my_list.append([n, rasters_csi_np_0[n][i][j]])
            my_list = sorted(my_list, key = sort_key, reverse=True)
            if len(my_list)>=5:
                top5_crops_num[j][i] = [(my_list[:5][k][0], my_list[:5][k][1]) for k in range(5)]

    # Computing indicators of representing TOP 5 crops in each cell 

    top5_ind = np.zeros((len(rasters_csi), 2160, 4320))
    for i in tqdm(range(2160)):
        for j in range(4320):
            if top5_crops_num[j][i] != 0:
                sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] + top5_crops_num[j][i][3][1] + top5_crops_num[j][i][4][1]
                top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][3][0]][i][j] = top5_crops_num[j][i][3][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][4][0]][i][j] = top5_crops_num[j][i][4][1]/sum_top5_csi

    # Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

    top5_crops_num_copy = top5_crops_num.copy()
    top5_crops_num_copy = (top5_crops_num_copy == 0)*1
    x = top5_crops_num_copy.astype('float64')
    x = x.to_numpy()

    for i in range(2160):
        for j in range(4320):
            if x[i][j]==1:
                x[i][j]=np.nan
            else: 
                x[i][j]=1

    # Adding 'No data' values to our indicators 

    for i in range(len(rasters_csi)):
        top5_ind[i] = x * top5_ind[i]

    # Saving indeces of representing TOP 5 crops in each cell 

    env.workspace = csi_path

    for i in tqdm(range(len(rasters_csi))):
        out = numpy_to_raster(top5_ind[i])
        out.save(tables_path + '\\A3\\A3_2\\' + continent + '\\csi_top5_ind\\' + rasters_csi[i])

    # Calculating the share of cells for which crop c is in TOP 5 across regions (for all crops)

    env.workspace = tables_path + '\\A3\\A3_2\\' + continent + '\\csi_top5_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A3\\A3_2\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A3\\A3_2\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A3\\A3_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A3\\A3_2\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A3\\A3_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A3\\A3_2\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A3_2_' + continent + '_READY.csv')

In [81]:
def computing_A3_2(rasters_csi, continent):
    env.workspace = tables_path + '\\A3\\A3_2\\' + continent + '\\csi_top5_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A3\\A3_2\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A3\\A3_2\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A3\\A3_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A3\\A3_2\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A3\\A3_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A3\\A3_2\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A3_2_' + continent + '_READY.csv')

In [82]:
computing_A3_2(rasters_csi = saf_rasters_csi, continent = 'saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  3.10it/s]


In [83]:
computing_A3_2(rasters_csi = naf_rasters_csi, continent = 'naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.86it/s]


In [84]:
computing_A3_2(rasters_csi = eu_rasters_csi, continent = 'eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:07<00:00,  2.27it/s]


In [85]:
computing_A3_2(rasters_csi = asia_rasters_csi, continent = 'asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:09<00:00,  2.80it/s]


In [86]:
computing_A3_2(rasters_csi = oceania_rasters_csi, continent = 'oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.48it/s]


In [87]:
computing_A3_2(rasters_csi = america_rasters_csi, continent = 'america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  2.90it/s]


In [53]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A3_2_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A3_2_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A3_2_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A3_2_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A3_2_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A3_2_america_READY.csv', sep = ';', decimal=',')

In [54]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [55]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A3_2'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A3_2']].to_excel(tables_path + '\\READY_country_level\\A3_2_READY.xlsx')

## A3_3

In [ ]:
# Finding out which crops make the TOP 5 in each cell

top5_crops_num = np.zeros((2160, 4320))
top5_crops_num = pd.DataFrame(top5_crops_num)
top5_crops_num = top5_crops_num.astype(object)

for i in tqdm(range(2160)):
    for j in range(4320):
        my_list = []
        for n in range(46):
            if rasters_csi_np_0[n][i][j] != 0:
                my_list.append([n, rasters_csi_np_0[n][i][j]])
        my_list = sorted(my_list, key = sort_key, reverse=True)
        if len(my_list)>=5:
            inds = [(my_list[:5][k][0], my_list[:5][k][1]) for k in range(5)]
            res = []
            for m in inds:
                if rasters_si_ind_np[m[0]][i][j] == 1:
                    res.append(m)
            top5_crops_num[j][i] = res

In [ ]:
# Computing indicators of representing TOP 5 crops in each cell 

top5_ind = np.zeros((46, 2160, 4320))
for i in tqdm(range(2160)):
    for j in range(4320):
        if top5_crops_num[j][i] != 0:
            if len(top5_crops_num[j][i])==5:
                sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] + top5_crops_num[j][i][3][1] + top5_crops_num[j][i][4][1]
                top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][3][0]][i][j] = top5_crops_num[j][i][3][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][4][0]][i][j] = top5_crops_num[j][i][4][1]/sum_top5_csi
            elif len(top5_crops_num[j][i])==4:
                sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] + top5_crops_num[j][i][3][1] 
                top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][3][0]][i][j] = top5_crops_num[j][i][3][1]/sum_top5_csi
            elif len(top5_crops_num[j][i])==3:
                sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] 
                top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
            elif len(top5_crops_num[j][i])==2:
                sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] 
                top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
            elif len(top5_crops_num[j][i])==1:
                sum_top5_csi = top5_crops_num[j][i][0][1]
                top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi             

In [ ]:
# Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

top5_crops_num_copy = top5_crops_num.copy()
top5_crops_num_copy = (top5_crops_num_copy == 0)*1
x = top5_crops_num_copy.astype('float64')
x = x.to_numpy()

for i in range(2160):
    for j in range(4320):
        if x[i][j]==1:
            x[i][j]=np.nan
        else: 
            x[i][j]=1

In [ ]:
env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))

In [ ]:
# Adding 'No data' values to our indicators 

for i in range(46):
    top5_ind[i] = x * top5_ind[i] * cutoff_constraint_np

# Saving indeces of representing TOP 5 crops in each cell 

env.workspace = csi_path

for i in tqdm(range(46)):
    out = numpy_to_raster(top5_ind[i])
    out.save(tables_path + '\\A3\\A3_3\\csi_top5_ind\\' + rasters_csi[i])

In [29]:
# Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

env.workspace = tables_path + '\\A3\\A3_3\\csi_top5_ind'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A3\\A3_3\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [25:59<00:00, 33.90s/it]


In [30]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A3\\A3_3\\share\\'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A3\\A3_3\\share_squared\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:19<00:00,  2.31it/s]


In [31]:
env.workspace = tables_path + '\\A3\\A3_3\\share_squared\\'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +"\\A3\\A3_3\\RESULTS", "DATA", "MAXIMUM")
tt(tables_path +"\\A3\\A3_3\\RESULTS", tables_path + '\\READY_country_level', 'A3_3_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A3_3_READY.csv'>

## A3_4

In [ ]:
def computing_A3_4(rasters_csi, rasters_si, continent):

    env.workspace = csi_path

    rasters_csi_np_0 = []
    for n in range(len(rasters_csi)):
        rasters_csi_np_0.append(raster_to_numpy_0(rasters_csi[n]))

    # Creating indicator-rasters for all crops which show whether SI in the specific cell is higher than 0.1 cutoff or not

    env.workspace = dtb_path

    rasters_si_ind_np = []
    for i in tqdm(range(len(rasters_csi))):
        rasters_si_ind_np.append((raster_to_numpy(Float(rasters_si[i]))>=1000)*1)

    # Finding out which crops make the TOP 5 in each cell

    top5_crops_num = np.zeros((2160, 4320))
    top5_crops_num = pd.DataFrame(top5_crops_num)
    top5_crops_num = top5_crops_num.astype(object)

    for i in tqdm(range(2160)):
        for j in range(4320):
            my_list = []
            for n in range(len(rasters_csi)):
                if rasters_csi_np_0[n][i][j] != 0:
                    my_list.append([n, rasters_csi_np_0[n][i][j]])
            my_list = sorted(my_list, key = sort_key, reverse=True)
            if len(my_list)>=5:
                inds = [(my_list[:5][k][0], my_list[:5][k][1]) for k in range(5)]
                res = []
                for m in inds:
                    if rasters_si_ind_np[m[0]][i][j] == 1:
                        res.append(m)
                top5_crops_num[j][i] = res

    # Computing indicators of representing TOP 5 crops in each cell 

    top5_ind = np.zeros((len(rasters_csi), 2160, 4320))
    for i in tqdm(range(2160)):
        for j in range(4320):
            if top5_crops_num[j][i] != 0:
                if len(top5_crops_num[j][i])==5:
                    sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] + top5_crops_num[j][i][3][1] + top5_crops_num[j][i][4][1]
                    top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                    top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                    top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
                    top5_ind[top5_crops_num[j][i][3][0]][i][j] = top5_crops_num[j][i][3][1]/sum_top5_csi
                    top5_ind[top5_crops_num[j][i][4][0]][i][j] = top5_crops_num[j][i][4][1]/sum_top5_csi
                elif len(top5_crops_num[j][i])==4:
                    sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] + top5_crops_num[j][i][3][1] 
                    top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                    top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                    top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
                    top5_ind[top5_crops_num[j][i][3][0]][i][j] = top5_crops_num[j][i][3][1]/sum_top5_csi
                elif len(top5_crops_num[j][i])==3:
                    sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] + top5_crops_num[j][i][2][1] 
                    top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                    top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                    top5_ind[top5_crops_num[j][i][2][0]][i][j] = top5_crops_num[j][i][2][1]/sum_top5_csi
                elif len(top5_crops_num[j][i])==2:
                    sum_top5_csi = top5_crops_num[j][i][0][1] + top5_crops_num[j][i][1][1] 
                    top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi
                    top5_ind[top5_crops_num[j][i][1][0]][i][j] = top5_crops_num[j][i][1][1]/sum_top5_csi 
                elif len(top5_crops_num[j][i])==1:
                    sum_top5_csi = top5_crops_num[j][i][0][1]
                    top5_ind[top5_crops_num[j][i][0][0]][i][j] = top5_crops_num[j][i][0][1]/sum_top5_csi 


    # Creating the multiplicator to take into account 'No Data' and zero values in our indicators 

    top5_crops_num_copy = top5_crops_num.copy()
    top5_crops_num_copy = (top5_crops_num_copy == 0)*1
    x = top5_crops_num_copy.astype('float64')
    x = x.to_numpy()

    for i in range(2160):
        for j in range(4320):
            if x[i][j]==1:
                x[i][j]=np.nan
            else: 
                x[i][j]=1

    env.workspace = dtb_path

    # Defining maximum SI in each cell
    max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

    # Avoiding cells where maximum SI <= 0.1
    cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
    cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))

    # Adding 'No data' values to our indicators 

    for i in range(len(rasters_csi)):
        top5_ind[i] = x * top5_ind[i] * cutoff_constraint_np

    # Saving indeces of representing TOP 5 crops in each cell 

    env.workspace = csi_path

    for i in tqdm(range(len(rasters_csi))):
        out = numpy_to_raster(top5_ind[i])
        out.save(tables_path + '\\A3\\A3_4\\' + continent + '\\csi_top5_ind\\' + rasters_csi[i])

    # Calculating the share of cells for which crop c is in TOP 3 across regions (for all crops)

    env.workspace = tables_path + '\\A3\\A3_4\\' + continent + '\\csi_top5_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A3\\A3_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A3\\A3_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A3\\A3_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A3\\A3_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A3\\A3_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A3\\A3_4\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A3_4_' + continent + '_READY.csv')

In [90]:
def computing_A3_4(rasters_csi, rasters_si, continent):
    env.workspace = tables_path + '\\A3\\A3_4\\' + continent + '\\csi_top5_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A3\\A3_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A3\\A3_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A3\\A3_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A3\\A3_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A3\\A3_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A3\\A3_4\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A3_4_' + continent + '_READY.csv')

In [91]:
computing_A3_4(rasters_csi=saf_rasters_csi, rasters_si = saf_rasters_si, continent='saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  3.30it/s]


In [92]:
computing_A3_4(rasters_csi=naf_rasters_csi, rasters_si = naf_rasters_si, continent='naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.58it/s]


In [93]:
computing_A3_4(rasters_csi=asia_rasters_csi, rasters_si = asia_rasters_si, continent='asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:17<00:00,  1.51it/s]


In [94]:
computing_A3_4(rasters_csi=eu_rasters_csi, rasters_si = eu_rasters_si, continent='eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.65it/s]


In [95]:
computing_A3_4(rasters_csi=america_rasters_csi, rasters_si = america_rasters_si, continent='america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.71it/s]


In [96]:
computing_A3_4(rasters_csi=oceania_rasters_csi, rasters_si = oceania_rasters_si, continent='oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.45it/s]


In [56]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A3_4_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A3_4_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A3_4_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A3_4_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A3_4_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A3_4_america_READY.csv', sep = ';', decimal=',')

In [57]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [58]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A3_4'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A3_4']].to_excel(tables_path + '\\READY_country_level\\A3_4_READY.xlsx')

## A4_1

In [ ]:
env.workspace = csi_path

# Defining maximum CSI in each cell
csi_mean_among_all_cells = Float(arcpy.sa.CellStatistics(rasters_csi, 'MEAN', 'DATA'))
csi_mean_among_all_cells.save(tables_path + '\\A4\\A4_1\\csi_mean')

In [ ]:
for raster in rasters_csi:
    out = Float(GreaterThan(raster, csi_mean_among_all_cells))
    out.save(tables_path + '\\A4\\A4_1\\csi_mean_ind\\' + raster)

In [ ]:
env.workspace = tables_path + '\\A4\\A4_1\\csi_mean_ind\\'

all_crops_np = []
for i in range(46):
    all_crops_np.append(raster_to_numpy(rasters_csi[i]))

In [ ]:
mean_ind = np.zeros((46, 2160, 4320))

for i in tqdm(range(2160)):
    for j in range(4320):
        my_list = []
        for n in range(46):
            if all_crops_np[n][i][j] == 1:
                my_list.append([n, rasters_csi_np[n][i][j]])
        sum_mean_csi = 0
        for m in range(len(my_list)):       
            sum_mean_csi += my_list[m][1] 
        for m in range(len(my_list)):
            mean_ind[my_list[m][0]][i][j] = rasters_csi_np[my_list[m][0]][i][j]/sum_mean_csi        

In [ ]:
# Adding 'No data' values to our indicators 

no_data_np = np.zeros((2160, 4320))

for i in tqdm(range(2160)):
    for j in range(4320):
        if np.isnan(all_crops_np[0][i][j])==True:
            no_data_np[i][j] = np.nan
        else:
            no_data_np[i][j] = 1

# Saving indeces of representing Higher than MEAN crops in each cell

env.workspace = csi_path

for n in range(46):
    mean_ind[n] = mean_ind[n] * no_data_np
    out = numpy_to_raster(mean_ind[n])
    out.save(tables_path + '\\A4\\A4_1\\csi_mean_ind\\' + rasters_csi[n])

In [32]:
# 

env.workspace = tables_path + '\\A4\\A4_1\\csi_mean_ind\\'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A4\\A4_1\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [25:56<00:00, 33.83s/it]


In [33]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A4\\A4_1\\share\\'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A4\\A4_1\\share_squared\\' + raster)

env.workspace = tables_path + '\\A4\\A4_1\\share_squared\\'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +'\\A4\\A4_1\\RESULTS', "DATA", "MAXIMUM")
tt(tables_path +'\\A4\\A4_1\\RESULTS', tables_path + '\\READY_country_level', 'A4_1_READY.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:27<00:00,  1.64it/s]


<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A4_1_READY.csv'>

## A4_2

In [ ]:
def computing_A4_2(rasters_csi, continent):

    env.workspace = csi_path

    # Defining maximum CSI in each cell
    csi_mean_among_all_cells = Float(arcpy.sa.CellStatistics(rasters_csi, 'MEAN', 'DATA'))

    for raster in rasters_csi:
        out = Float(GreaterThan(raster, csi_mean_among_all_cells))
        out.save(tables_path + '\\A4\\A4_2\\' + continent + '\\csi_mean_ind\\' + raster)

    env.workspace = tables_path + '\\A4\\A4_2\\' + continent + '\\csi_mean_ind\\'

    all_crops_np = []
    for i in range(len(rasters_csi)):
        all_crops_np.append(raster_to_numpy(rasters_csi[i]))


    mean_ind = np.zeros((len(rasters_csi), 2160, 4320))

    for i in tqdm(range(2160)):
        for j in range(4320):
            my_list = []
            for n in range(len(rasters_csi)):
                if all_crops_np[n][i][j] == 1:
                    my_list.append([n, rasters_csi_np[n][i][j]])
            sum_mean_csi = 0
            for m in range(len(my_list)):       
                sum_mean_csi += my_list[m][1]
            if sum_mean_csi > 0:
                for m in range(len(my_list)):
                    mean_ind[my_list[m][0]][i][j] = rasters_csi_np[my_list[m][0]][i][j]/sum_mean_csi 

    # Adding 'No data' values to our indicators 

    no_data_np = np.zeros((2160, 4320))

    for i in tqdm(range(2160)):
        for j in range(4320):
            if np.isnan(all_crops_np[0][i][j])==True:
                no_data_np[i][j] = np.nan
            else:
                no_data_np[i][j] = 1

    # Saving indeces of representing Higher than MEAN crops in each cell

    env.workspace = csi_path

    for n in range(len(rasters_csi)):
        mean_ind[n] = mean_ind[n] * no_data_np
        out = numpy_to_raster(mean_ind[n])
        out.save(tables_path + '\\A4\\A4_2\\' + continent + '\\csi_mean_ind\\' + rasters_csi[n])



    env.workspace = tables_path + '\\A4\\A4_2\\' + continent + '\\csi_mean_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A4\\A4_2\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A4\\A4_2\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A4\\A4_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A4\\A4_2\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A4\\A4_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A4\\A4_2\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A4_2_' + continent + '_READY.csv')

In [97]:
def computing_A4_2(rasters_csi, continent):
    env.workspace = tables_path + '\\A4\\A4_2\\' + continent + '\\csi_mean_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A4\\A4_2\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A4\\A4_2\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A4\\A4_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A4\\A4_2\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A4\\A4_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A4\\A4_2\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A4_2_' + continent + '_READY.csv')

In [98]:
computing_A4_2(rasters_csi = saf_rasters_csi, continent = 'saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.67it/s]


In [99]:
computing_A4_2(rasters_csi = naf_rasters_csi, continent = 'naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.82it/s]


In [100]:
computing_A4_2(rasters_csi = eu_rasters_csi, continent = 'eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.74it/s]


In [101]:
computing_A4_2(rasters_csi = asia_rasters_csi, continent = 'asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:09<00:00,  2.79it/s]


In [102]:
computing_A4_2(rasters_csi = oceania_rasters_csi, continent = 'oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.84it/s]


In [103]:
computing_A4_2(rasters_csi = america_rasters_csi, continent = 'america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.70it/s]


In [59]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A4_2_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A4_2_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A4_2_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A4_2_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A4_2_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A4_2_america_READY.csv', sep = ';', decimal=',')

In [60]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [61]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A4_2'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A4_2']].to_excel(tables_path + '\\READY_country_level\\A4_2_READY.xlsx')

## A4_3

In [ ]:
env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")

# Creating CSI rasters which satisfy the constraint:
# 1) max SI across all crops >= 0.1 (suitable cell for agriculture)

env.workspace = csi_path

for i in tqdm(range(46)):
    out = Raster(csi_path + '\\' + rasters_csi[i]) * Raster(cutoff_constraint)
    Float(out).save(tables_path + '\\A4\\A4_3\\rastersXind\\' + rasters_csi[i])

In [ ]:
env.workspace = tables_path + '\\A4\\A4_3\\rastersXind\\'

# Defining mean CSI in each cell
csi_mean_among_all_cells = Float(arcpy.sa.CellStatistics(rasters_csi, 'MEAN', 'DATA'))
csi_mean_among_all_cells.save(tables_path + '\\A4\\A4_3\\csi_mean')

for raster in rasters_csi:
    out = Float(GreaterThan(raster, csi_mean_among_all_cells))
    out.save(tables_path + '\\A4\\A4_3\\csi_mean_ind\\' + raster)

In [ ]:
env.workspace = tables_path + '\\A4\\A4_3\\csi_mean_ind\\'

all_crops_np = []
for i in range(46):
    all_crops_np.append(raster_to_numpy(rasters_csi[i]))

In [ ]:
mean_ind = np.zeros((46, 2160, 4320))

for i in tqdm(range(2160)):
    for j in range(4320):
        res = []
        for n in range(46):
            if all_crops_np[n][i][j] == 1:
                res.append([n, rasters_csi_np[n][i][j]])
        my_list = []
        for m in res:
             if rasters_si_ind_np[m[0]][i][j] == 1:
                    my_list.append(m)
                    
        sum_mean_csi = 0
        for m in range(len(my_list)):       
            sum_mean_csi += my_list[m][1] 
        for m in range(len(my_list)):
            mean_ind[my_list[m][0]][i][j] = rasters_csi_np[my_list[m][0]][i][j]/sum_mean_csi    

In [ ]:
# Adding 'No data' values to our indicators 

no_data_np = np.zeros((2160, 4320))

for i in tqdm(range(2160)):
    for j in range(4320):
        if np.isnan(all_crops_np[0][i][j])==True:
            no_data_np[i][j] = np.nan
        else:
            no_data_np[i][j] = 1

# Saving indeces of representing Higher than MEAN crops in each cell

env.workspace = csi_path

for n in range(46):
    mean_ind[n] = mean_ind[n] * no_data_np
    out = numpy_to_raster(mean_ind[n])
    out.save(tables_path + '\\A4\\A4_3\\csi_mean_ind\\' + rasters_csi[n])

In [34]:
# 

env.workspace = tables_path + '\\A4\\A4_3\\csi_mean_ind\\'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A4\\A4_3\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [26:22<00:00, 34.41s/it]


In [35]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A4\\A4_3\\share\\'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A4\\A4_3\\share_squared\\' + raster)

env.workspace = tables_path + '\\A4\\A4_3\\share_squared\\'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +'\\A4\\A4_3\\RESULTS', "DATA", "MAXIMUM")
tt(tables_path +'\\A4\\A4_3\\RESULTS', tables_path + '\\READY_country_level', 'A4_3_READY.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:18<00:00,  2.50it/s]


<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A4_3_READY.csv'>

## A4_4

In [ ]:
def computing_A4_4(rasters_csi, rasters_si, continent):

    env.workspace = tables_path + '\\A4\\A4_3\\rastersXind\\'

    # Defining mean CSI in each cell
    csi_mean_among_all_cells = Float(arcpy.sa.CellStatistics(rasters_csi, 'MEAN', 'DATA'))

    for raster in rasters_csi:
        out = Float(GreaterThan(raster, csi_mean_among_all_cells))
        out.save(tables_path + '\\A4\\A4_4\\' + continent + '\\csi_mean_ind\\' + raster)

    env.workspace = tables_path + '\\A4\\A4_4\\' + continent + '\\csi_mean_ind\\'

    all_crops_np = []
    for i in range(len(rasters_csi)):
        all_crops_np.append(raster_to_numpy(rasters_csi[i]))  

    # Creating indicator-rasters for all crops which show whether SI in the specific cell is higher than 0.1 cutoff or not

    env.workspace = dtb_path

    rasters_si_ind_np = []
    for i in tqdm(range(len(rasters_csi))):
        rasters_si_ind_np.append((raster_to_numpy(Float(rasters_si[i]))>=1000)*1)


    mean_ind = np.zeros((len(rasters_csi), 2160, 4320))

    for i in tqdm(range(2160)):
        for j in range(4320):
            res = []
            for n in range(len(rasters_csi)):
                if all_crops_np[n][i][j] == 1:
                    res.append([n, rasters_csi_np[n][i][j]])
            my_list = []
            for m in res:
                 if rasters_si_ind_np[m[0]][i][j] == 1:
                        my_list.append(m)

            sum_mean_csi = 0
            for m in range(len(my_list)):       
                sum_mean_csi += my_list[m][1] 
            if sum_mean_csi > 0:    
                for m in range(len(my_list)):
                    mean_ind[my_list[m][0]][i][j] = rasters_csi_np[my_list[m][0]][i][j]/sum_mean_csi     

    # Adding 'No data' values to our indicators 

    no_data_np = np.zeros((2160, 4320))

    for i in tqdm(range(2160)):
        for j in range(4320):
            if np.isnan(all_crops_np[0][i][j])==True:
                no_data_np[i][j] = np.nan
            else:
                no_data_np[i][j] = 1

    # Saving indeces of representing Higher than MEAN crops in each cell

    env.workspace = csi_path

    for n in range(len(rasters_csi)):
        mean_ind[n] = mean_ind[n] * no_data_np
        out = numpy_to_raster(mean_ind[n])
        out.save(tables_path + '\\A4\\A4_4\\' + continent + '\\csi_mean_ind\\' + rasters_csi[n])   


    env.workspace = tables_path + '\\A4\\A4_4\\' + continent + '\\csi_mean_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A4\\A4_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A4\\A4_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A4\\A4_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A4\\A4_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A4\\A4_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A4\\A4_4\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A4_4_' + continent + '_READY.csv')

In [104]:
def computing_A4_4(rasters_csi, rasters_si, continent):
    env.workspace = tables_path + '\\A4\\A4_4\\' + continent + '\\csi_mean_ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A4\\A4_4\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A4\\A4_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A4\\A4_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A4\\A4_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A4\\A4_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A4\\A4_4\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A4_4_' + continent + '_READY.csv')

In [105]:
computing_A4_4(rasters_csi=saf_rasters_csi, rasters_si = saf_rasters_si, continent='saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.73it/s]


In [106]:
computing_A4_4(rasters_csi=naf_rasters_csi, rasters_si = naf_rasters_si, continent='naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.81it/s]


In [107]:
computing_A4_4(rasters_csi=asia_rasters_csi, rasters_si = asia_rasters_si, continent='asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:09<00:00,  2.73it/s]


In [108]:
computing_A4_4(rasters_csi=eu_rasters_csi, rasters_si = eu_rasters_si, continent='eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.79it/s]


In [109]:
computing_A4_4(rasters_csi=america_rasters_csi, rasters_si = america_rasters_si, continent='america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.71it/s]


In [110]:
computing_A4_4(rasters_csi=oceania_rasters_csi, rasters_si = oceania_rasters_si, continent='oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.11it/s]


In [62]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A4_4_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A4_4_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A4_4_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A4_4_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A4_4_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A4_4_america_READY.csv', sep = ';', decimal=',')

In [63]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [64]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A4_4'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A4_4']].to_excel(tables_path + '\\READY_country_level\\A4_4_READY.xlsx')

## A5_1

In [ ]:
ind = np.zeros((46, 2160, 4320))

for i in tqdm(range(2160)):
    for j in range(4320):
        sum_csi = 0
        for n in range(46):
            if rasters_csi_np[n][i][j]>=0:
                sum_csi += rasters_csi_np[n][i][j]
        if sum_csi > 0:
            for n in range(46):
                ind[n][i][j] = rasters_csi_np[n][i][j] / sum_csi

In [ ]:
# Adding 'No data' values to our indicators 

no_data_np = np.zeros((2160, 4320))

x = raster_to_numpy(tables_path + '\\A1\\A1_1\\csi_max')
for i in tqdm(range(2160)):
    for j in range(4320):
        if np.isnan(x[i][j])==True:
            no_data_np[i][j] = np.nan
        else:
            no_data_np[i][j] = 1

# Saving indeces of representing Higher than MEAN crops in each cell

env.workspace = csi_path

for n in range(46):
    ind[n] = ind[n] * no_data_np
    out = numpy_to_raster(ind[n])
    out.save(tables_path + '\\A5\\A5_1\\ind\\' + rasters_csi[n])

In [36]:
env.workspace = tables_path + '\\A5\\A5_1\\ind\\'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A5\\A5_1\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [26:08<00:00, 34.09s/it]


In [37]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A5\\A5_1\\share\\'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A5\\A5_1\\share_squared\\' + raster)

env.workspace = tables_path + '\\A5\\A5_1\\share_squared\\'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +'\\A5\\A5_1\\RESULTS', "DATA", "MAXIMUM")
tt(tables_path +'\\A5\\A5_1\\RESULTS', tables_path + '\\READY_country_level', 'A5_1_READY.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:18<00:00,  2.53it/s]


<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A5_1_READY.csv'>

## A5_2

In [ ]:
def computing_A5_2 (rasters_csi, continent): 

    # Making np.arrays with CSI data for all crops
    
    env.workspace = csi_path

    rasters_csi_np = []
    for n in range(len(rasters_csi)):
        rasters_csi_np.append(raster_to_numpy(rasters_csi[n]))

    ind = np.zeros((len(rasters_csi), 2160, 4320))

    for i in tqdm(range(2160)):
        for j in range(4320):
            sum_csi = 0
            for n in range(len(rasters_csi)):
                if rasters_csi_np[n][i][j]>=0:
                    sum_csi += rasters_csi_np[n][i][j]
            if sum_csi > 0:
                for n in range(len(rasters_csi)):
                    ind[n][i][j] = rasters_csi_np[n][i][j] / sum_csi   


    # Adding 'No data' values to our indicators 

    no_data_np = np.zeros((2160, 4320))

    x = raster_to_numpy(tables_path + '\\A1\\A1_1\\csi_max')
    for i in tqdm(range(2160)):
        for j in range(4320):
            if np.isnan(x[i][j])==True:
                no_data_np[i][j] = np.nan
            else:
                no_data_np[i][j] = 1

    # Saving indeces of representing Higher than MEAN crops in each cell

    env.workspace = csi_path

    for n in range(len(rasters_csi)):
        ind[n] = ind[n] * no_data_np
        out = numpy_to_raster(ind[n])
        out.save(tables_path + '\\A5\\A5_2\\' + continent + '\\ind\\' + rasters_csi[n])

    env.workspace = tables_path + '\\A5\\A5_2\\' + continent + '\\ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A5\\A5_2\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A5\\A5_2\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A5\\A5_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A5\\A5_2\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A5\\A5_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A5\\A5_2\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A5_2_' + continent + '_READY.csv')

In [111]:
def computing_A5_2 (rasters_csi, continent):     
    env.workspace = tables_path + '\\A5\\A5_2\\' + continent + '\\ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A5\\A5_2\\' + continent + '\\share\\' + raster)

    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A5\\A5_2\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A5\\A5_2\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A5\\A5_2\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A5\\A5_2\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A5\\A5_2\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A5_2_' + continent + '_READY.csv')

In [112]:
computing_A5_2(rasters_csi = saf_rasters_csi, continent = 'saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:04<00:00,  2.65it/s]


In [113]:
computing_A5_2(rasters_csi = naf_rasters_csi, continent = 'naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.72it/s]


In [114]:
computing_A5_2(rasters_csi = eu_rasters_csi, continent = 'eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:06<00:00,  2.72it/s]


In [115]:
computing_A5_2(rasters_csi = asia_rasters_csi, continent = 'asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:10<00:00,  2.68it/s]


In [116]:
computing_A5_2(rasters_csi = oceania_rasters_csi, continent = 'oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.72it/s]


In [118]:
computing_A5_2(rasters_csi = america_rasters_csi, continent = 'america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.77it/s]


In [65]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A5_2_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A5_2_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A5_2_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A5_2_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A5_2_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A5_2_america_READY.csv', sep = ';', decimal=',')

In [66]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [67]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A5_2'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A5_2']].to_excel(tables_path + '\\READY_country_level\\A5_2_READY.xlsx')

## A5_3

In [ ]:
rasters_si_ind = []
for i in range(46):
    rasters_si_ind.append((rasters_si_np[i]>=1000)*1)

In [ ]:
ind = np.zeros((46, 2160, 4320))

for i in tqdm(range(2160)):
    for j in range(4320):
        sum_csi = 0
        for n in range(46):
            if rasters_csi_np[n][i][j]>=0:
                if rasters_si_ind[n][i][j]==1:
                    sum_csi += rasters_csi_np[n][i][j]
        if sum_csi > 0:
            for n in range(46):
                if rasters_si_ind[n][i][j]==1:
                    ind[n][i][j] = rasters_csi_np[n][i][j] / sum_csi

In [ ]:
# Adding 'No data' values to our indicators 

no_data_np = np.zeros((2160, 4320))

x = raster_to_numpy(tables_path + '\\A1\\A1_1\\csi_max')
for i in tqdm(range(2160)):
    for j in range(4320):
        if np.isnan(x[i][j])==True:
            no_data_np[i][j] = np.nan
        else:
            no_data_np[i][j] = 1
            
            
env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))

# Saving indeces of representing Higher than MEAN crops in each cell

env.workspace = csi_path

for n in range(46):
    ind[n] = ind[n] * no_data_np * cutoff_constraint_np
    out = numpy_to_raster(ind[n])
    out.save(tables_path + '\\A5\\A5_3\\ind\\' + rasters_csi[n])

In [38]:
env.workspace = tables_path + '\\A5\\A5_3\\ind\\'

for raster in tqdm(rasters_csi):
    out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
    out.save(tables_path + '\\A5\\A5_3\\share\\' + raster)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [27:45<00:00, 36.21s/it]


In [39]:
# Final steps to compute ready indeces

env.workspace = tables_path + '\\A5\\A5_3\\share\\'

for raster in tqdm(rasters_csi):
    #calculating squares 
    sq = Square(raster)
    sq.save(tables_path + '\\A5\\A5_3\\share_squared\\' + raster)

env.workspace = tables_path + '\\A5\\A5_3\\share_squared\\'

out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +'\\A5\\A5_3\\RESULTS', "DATA", "MAXIMUM")
tt(tables_path +'\\A5\\A5_3\\RESULTS', tables_path + '\\READY_country_level', 'A5_3_READY.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:19<00:00,  2.35it/s]


<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\A5_3_READY.csv'>

## A5_4

In [ ]:
def computing_A5_4(rasters_csi, rasters_si, continent):

    # Creating indicator-rasters for all crops which show whether SI in the specific cell is higher than 0.1 cutoff or not

    env.workspace = dtb_path

    rasters_si_ind = []
    for i in tqdm(range(len(rasters_csi))):
        rasters_si_ind.append((raster_to_numpy(Float(rasters_si[i]))>=1000)*1)

    ind = np.zeros((len(rasters_csi), 2160, 4320))

    for i in tqdm(range(2160)):
        for j in range(4320):
            sum_csi = 0
            for n in range(len(rasters_csi)):
                if rasters_csi_np[n][i][j]>=0:
                    if rasters_si_ind[n][i][j]==1:
                        sum_csi += rasters_csi_np[n][i][j]
            if sum_csi > 0:
                for n in range(len(rasters_csi)):
                    if rasters_si_ind[n][i][j]==1:
                        ind[n][i][j] = rasters_csi_np[n][i][j] / sum_csi


    # Adding 'No data' values to our indicators 

    no_data_np = np.zeros((2160, 4320))

    x = raster_to_numpy(tables_path + '\\A1\\A1_1\\csi_max')
    for i in tqdm(range(2160)):
        for j in range(4320):
            if np.isnan(x[i][j])==True:
                no_data_np[i][j] = np.nan
            else:
                no_data_np[i][j] = 1


    env.workspace = dtb_path

    # Defining maximum SI in each cell
    max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))

    # Avoiding cells where maximum SI <= 0.1
    cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")
    cutoff_constraint_np = raster_to_numpy(Float(cutoff_constraint))

    # Saving indeces of representing Higher than MEAN crops in each cell

    env.workspace = csi_path

    for n in range(len(rasters_csi)):
        ind[n] = ind[n] * no_data_np * cutoff_constraint_np
        out = numpy_to_raster(ind[n])
        out.save(tables_path + '\\A5\\A5_4\\' + continent + '\\ind\\' + rasters_csi[n])


    env.workspace = tables_path + '\\A5\\A5_4\\' + continent + '\\ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A5\\A5_4\\' + continent + '\\share\\' + raster)


    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A5\\A5_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A5\\A5_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A5\\A5_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A5\\A5_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A5\\A5_4\\' + continent + '\\RESULTS', tables_path + '\\READY', 'A5_4_' + continent + '_READY.csv')

In [119]:
def computing_A5_4(rasters_csi, rasters_si, continent):
    env.workspace = tables_path + '\\A5\\A5_4\\' + continent + '\\ind\\'

    for raster in tqdm(rasters_csi):
        out = ZonalStatistics(regions_shape_path, VAR, raster, "MEAN", "DATA")
        out.save(tables_path + '\\A5\\A5_4\\' + continent + '\\share\\' + raster)


    # Final steps to compute ready indeces

    env.workspace = tables_path + '\\A5\\A5_4\\' + continent + '\\share\\'

    for raster in tqdm(rasters_csi):
        #calculating squares 
        sq = Square(raster)
        sq.save(tables_path + '\\A5\\A5_4\\' + continent + '\\share_squared\\' + raster)

    env.workspace = tables_path + '\\A5\\A5_4\\' + continent + '\\share_squared\\'

    out = Float(arcpy.sa.CellStatistics(rasters_csi, 'SUM', 'DATA'))
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                            tables_path +'\\A5\\A5_4\\' + continent + '\\RESULTS', "DATA", "MAXIMUM")
    tt(tables_path +'\\A5\\A5_4\\' + continent + '\\RESULTS', tables_path + '\\READY_country_level', 'A5_4_' + continent + '_READY.csv')

In [120]:
computing_A5_4(rasters_csi=saf_rasters_csi, rasters_si = saf_rasters_si, continent='saf')

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  2.82it/s]


In [121]:
computing_A5_4(rasters_csi=naf_rasters_csi, rasters_si = naf_rasters_si, continent='naf')

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:06<00:00,  2.75it/s]


In [122]:
computing_A5_4(rasters_csi=asia_rasters_csi, rasters_si = asia_rasters_si, continent='asia')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:09<00:00,  2.72it/s]


In [123]:
computing_A5_4(rasters_csi=eu_rasters_csi, rasters_si = eu_rasters_si, continent='eu')

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  2.96it/s]


In [124]:
computing_A5_4(rasters_csi=america_rasters_csi, rasters_si = america_rasters_si, continent='america')

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:05<00:00,  2.68it/s]


In [125]:
computing_A5_4(rasters_csi=oceania_rasters_csi, rasters_si = oceania_rasters_si, continent='oceania')

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.80it/s]


In [68]:
eu = pd.read_csv(tables_path + '\\READY_country_level\\A5_4_eu_READY.csv', sep = ';', decimal=',')
saf = pd.read_csv(tables_path + '\\READY_country_level\\A5_4_saf_READY.csv', sep = ';', decimal=',')
naf = pd.read_csv(tables_path + '\\READY_country_level\\A5_4_naf_READY.csv', sep = ';', decimal=',')
oceania = pd.read_csv(tables_path + '\\READY_country_level\\A5_4_oceania_READY.csv', sep = ';', decimal=',')
asia = pd.read_csv(tables_path + '\\READY_country_level\\A5_4_asia_READY.csv', sep = ';', decimal=',')
america = pd.read_csv(tables_path + '\\READY_country_level\\A5_4_america_READY.csv', sep = ';', decimal=',')

In [69]:
lst = [eu, saf, naf, oceania, asia, america]
lst_n = ['eu', 'saf', 'naf', 'oceania', 'asia', 'america']
for i in range(len(lst)): 
    lst[i].rename(columns = {"MAX": lst_n[i]}, inplace=True)

In [70]:
res = asia.merge(eu[[VAR, 'eu']], how= 'left', on = VAR).merge(america[[VAR, 'america']], how= 'left', on = VAR).merge(saf[[VAR, 'saf']], how= 'left', on = VAR).merge(naf[[VAR, 'naf']], how= 'left', on = VAR).merge(oceania[[VAR, 'oceania']], how= 'left', on = VAR)
res.fillna(0, inplace=True)
res = res.merge(regions_pre1500, on=VAR, how='left').drop(['rcode', 'REGIONID'], axis=1).drop_duplicates().reset_index(drop=True)
res['A5_4'] = res['Subsaharan Africa'] * res['saf'] + res['North Africa'] * res['naf'] + res['Europe'] * res['eu'] + res['Asia'] * res['asia'] + res['Oceania'] * res['oceania'] + res['America'] * res['america']  
res[[VAR, 'COUNT', 'AREA', 'A5_4']].to_excel(tables_path + '\\READY_country_level\\A5_4_READY.xlsx')

## B0_1

In [15]:
env.workspace = dtb_path

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[0],
                        tables_path +'\\B0\\B0_1\\RESULTS', "DATA", "STD")
tt(tables_path +'\\B0\\B0_1\\RESULTS', tables_path + '\\B0\\help_tables', 'B0_1_ready.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\B0\\help_tables\\B0_1_ready.csv'>

In [16]:
df = pd.read_csv(tables_path + "\\B0\\help_tables\\B0_1_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["STD"]/10000

for i in tqdm(range(1, 46)):
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[i],
                        tables_path +'\\B0\\B0_1\\RESULTS', "DATA", "STD")
    tt(tables_path +'\\B0\\B0_1\\RESULTS', tables_path + '\\B0\\help_tables', 'B0_1_ready' + str(i) + '.csv')
    df1 = pd.read_csv(tables_path + "\\B0\\help_tables\\B0_1_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["STD"]/10000
    df = df.merge(df1[["COUNTRY", rasters_si[i]]], on = "COUNTRY")
    
df.drop(["OID_", "ZONE_CODE", "STD"], axis=1, inplace=True)

output_path = tables_path + '\\READY_country_level\\B0_1_READY.xlsx'
df.to_excel(output_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [48:39<00:00, 64.87s/it]


## B0_2

In [17]:
env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))
# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")

# Creating SI rasters which satisfy the constraint:
# 1) max SI across all crops >= 0.1 (suitable cell for agriculture)

for i in tqdm(range(46)):
    out = Raster(rasters_si[i]) * Raster(cutoff_constraint)
    Float(out).save(tables_path + '\\B0\\B0_2\\rasters_si\\' + rasters_si[i])

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:29<00:00,  1.58it/s]


In [18]:
env.workspace = tables_path + '\\B0\\B0_2\\rasters_si\\'

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[0],
                        tables_path +'\\B0\\B0_2\\RESULTS', "DATA", "STD")
tt(tables_path +'\\B0\\B0_2\\RESULTS', tables_path + '\\B0\\help_tables', 'B0_2_ready.csv')

df = pd.read_csv(tables_path + "\\B0\\help_tables\\B0_2_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["STD"]/10000

for i in tqdm(range(1, 46)):
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[i],
                        tables_path +'\\B0\\B0_2\\RESULTS', "DATA", "STD")
    tt(tables_path +'\\B0\\B0_2\\RESULTS', tables_path + '\\B0\\help_tables', 'B0_2_ready' + str(i) + '.csv')
    df1 = pd.read_csv(tables_path + "\\B0\\help_tables\\B0_2_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["STD"]/10000
    df = df.merge(df1[["COUNTRY", rasters_si[i]]], on = "COUNTRY")
    
df.drop(["OID_", "ZONE_CODE", "STD"], axis=1, inplace=True)
output_path = tables_path + '\\READY_country_level\\B0_2_READY.xlsx'
df.to_excel(output_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [29:50<00:00, 39.78s/it]


## B1_1

In [ ]:
# Computing weights

csi_max_among_all_cells = Raster(tables_path + '\\A1\\A1_1\\csi_max')
sum_of_maxcsi = ZonalStatistics(regions_shape_path, VAR, csi_max_among_all_cells, "SUM", "DATA")
w_s = Raster(csi_max_among_all_cells) / Raster(sum_of_maxcsi)

# Adjusting SI rasters with weights 

env.workspace = dtb_path
for i in range(len(rasters_si)):
    out = Raster(rasters_si[i]) * Raster(w_s)
    out.save(tables_path + '\\B1\\B1_1\\rasters_si\\' + rasters_si[i])

In [ ]:
env.workspace = tables_path + '\\B1\\B1_1\\rasters_si\\'

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[0],
                        tables_path +'\\B1\\B1_1\\RESULTS', "DATA", "STD")
tt(tables_path +'\\B1\\B1_1\\RESULTS', tables_path + '\\B1\\help_tables', 'B1_1_ready.csv')

df = pd.read_csv(tables_path + "\\B1\\help_tables\\B1_1_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["STD"]/10000

for i in tqdm(range(1, 46)):
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[i],
                        tables_path +'\\B1\\B1_1\\RESULTS', "DATA", "STD")
    tt(tables_path +'\\B1\\B1_1\\RESULTS', tables_path + '\\B1\\help_tables', 'B1_1_ready' + str(i) + '.csv')
    df1 = pd.read_csv(tables_path + "\\B1\\help_tables\\B1_1_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["STD"]/10000
    df = df.merge(df1[["REGIONID", rasters_si[i]]], on = "REGIONID")
    
df.drop(["OID_", "ZONE_CODE", "STD"], axis=1, inplace=True)
output_path = tables_path + '\\READY\\B1_1_READY.xlsx'
df.to_excel(output_path)

## B1b_1

In [11]:
# Computing weights

csi_max_among_all_cells = Raster(tables_path + '\\A1\\A1_1\\csi_max')
sum_of_maxcsi = ZonalStatistics(regions_shape_path, VAR, csi_max_among_all_cells, "SUM", "DATA")
w_s = Raster(csi_max_among_all_cells) / Raster(sum_of_maxcsi)

# Adjusting SI rasters with weights 

env.workspace = dtb_path
for i in range(len(rasters_si)):
    out = Raster(rasters_si[i]) * Raster(w_s)
    out.save(tables_path + '\\B1\\B1_1\\rasters_si\\' + rasters_si[i])

In [13]:
env.workspace = tables_path + '\\B1\\B1_1\\rasters_si\\'
sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[0], "SUM", "DATA")

env.workspace = dtb_path
si_minus_sum_w_si = Raster(rasters_si[0]) - Raster(sum_w_si)

square = Square(si_minus_sum_w_si)
under_root = Raster(w_s) * Raster(square)
out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                        tables_path +'\\B1\\B1b_1\\RESULTS', "DATA", "MINIMUM")

tt(tables_path +'\\B1\\B1b_1\\RESULTS', tables_path + '\\B1\\help_tables', 'B1b_1_ready.csv')

df = pd.read_csv(tables_path + "\\B1\\help_tables\\B1b_1_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["MIN"]/10000

In [14]:
for i in tqdm(range(1, 46)):
    env.workspace = tables_path + '\\B1\\B1_1\\rasters_si\\'
    sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[i], "SUM", "DATA")

    env.workspace = dtb_path
    si_minus_sum_w_si = Raster(rasters_si[i]) - Raster(sum_w_si)

    square = Square(si_minus_sum_w_si)
    under_root = Raster(w_s) * Raster(square)
    out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                            tables_path +'\\B1\\B1b_1\\RESULTS', "DATA", "MINIMUM")

    tt(tables_path +'\\B1\\B1b_1\\RESULTS', tables_path + '\\B1\\help_tables', 'B1b_1_ready' + str(i) + '.csv')

    
    df1 = pd.read_csv(tables_path + "\\B1\\help_tables\\B1b_1_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["MIN"]/10000
    df = df.merge(df1[["COUNTRY", rasters_si[i]]], on = "COUNTRY")
    
df.drop(["OID_", "ZONE_CODE", "MIN"], axis=1, inplace=True)
output_path = tables_path + '\\READY_country_level\\B1b_1_READY.xlsx'
df.to_excel(output_path)

100%|████████████████████████████████████████████████████████████████████████████████| 45/45 [1:08:19<00:00, 91.11s/it]


## B1_2

In [ ]:
# Computing weights

csi_max_among_all_cells = Raster(tables_path + '\\A1\\A1_1\\csi_max')
sum_of_maxcsi = ZonalStatistics(regions_shape_path, VAR, csi_max_among_all_cells, "SUM", "DATA")
w_s = Raster(csi_max_among_all_cells) / Raster(sum_of_maxcsi)

env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))
# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")

# Creating SI rasters which satisfy the constraint and are multiplied by weights:
# 1) max SI across all crops >= 0.1 (suitable cell for agriculture)

for i in tqdm(range(46)):
    out = Raster(rasters_si[i]) * Raster(cutoff_constraint) * Raster(w_s)
    Float(out).save(tables_path + '\\B1\\B1_2\\rasters_si\\' + rasters_si[i])

In [ ]:
env.workspace = tables_path + '\\B1\\B1_2\\rasters_si\\'

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[0],
                        tables_path +'\\B1\\B1_2\\RESULTS', "DATA", "STD")
tt(tables_path +'\\B1\\B1_2\\RESULTS', tables_path + '\\B1\\help_tables', 'B1_2_ready.csv')

df = pd.read_csv(tables_path + "\\B1\\help_tables\\B1_2_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["STD"]/10000

for i in tqdm(range(1, 46)):
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[i],
                        tables_path +'\\B1\\B1_2\\RESULTS', "DATA", "STD")
    tt(tables_path +'\\B1\\B1_2\\RESULTS', tables_path + '\\B1\\help_tables', 'B1_2_ready' + str(i) + '.csv')
    df1 = pd.read_csv(tables_path + "\\B1\\help_tables\\B1_2_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["STD"]/10000
    df = df.merge(df1[["REGIONID", rasters_si[i]]], on = "REGIONID")
    
df.drop(["OID_", "ZONE_CODE", "STD"], axis=1, inplace=True)
output_path = tables_path + '\\READY\\B1_2_READY.xlsx'
df.to_excel(output_path)

## B1b_2

In [15]:
# Computing weights

csi_max_among_all_cells = Raster(tables_path + '\\A1\\A1_1\\csi_max')
sum_of_maxcsi = ZonalStatistics(regions_shape_path, VAR, csi_max_among_all_cells, "SUM", "DATA")
w_s = Raster(csi_max_among_all_cells) / Raster(sum_of_maxcsi)

env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))
# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")

# Creating SI rasters which satisfy the constraint and are multiplied by weights:
# 1) max SI across all crops >= 0.1 (suitable cell for agriculture)

for i in tqdm(range(46)):
    out = Raster(rasters_si[i]) * Raster(cutoff_constraint) * Raster(w_s)
    Float(out).save(tables_path + '\\B1\\B1_2\\rasters_si\\' + rasters_si[i])

    
    
env.workspace = tables_path + '\\B1\\B1_2\\rasters_si\\'
sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[0], "SUM", "DATA")

env.workspace = dtb_path
si_minus_sum_w_si = Raster(rasters_si[0]) - Raster(sum_w_si)

square = Square(si_minus_sum_w_si)
under_root = Raster(w_s) * Raster(square)
out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                        tables_path +'\\B1\\B1b_2\\RESULTS', "DATA", "MINIMUM")
tt(tables_path +'\\B1\\B1b_2\\RESULTS', tables_path + '\\B1\\help_tables', 'B1b_2_ready.csv')
df = pd.read_csv(tables_path + "\\B1\\help_tables\\B1b_2_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["MIN"]/10000


for i in tqdm(range(1, 46)):
    env.workspace = tables_path + '\\B1\\B1_2\\rasters_si\\'
    sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[i], "SUM", "DATA")

    env.workspace = dtb_path
    si_minus_sum_w_si = Raster(rasters_si[i]) - Raster(sum_w_si)

    square = Square(si_minus_sum_w_si)
    under_root = Raster(w_s) * Raster(square)
    out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                            tables_path +'\\B1\\B1b_2\\RESULTS', "DATA", "MINIMUM")

    tt(tables_path +'\\B1\\B1b_2\\RESULTS', tables_path + '\\B1\\help_tables', 'B1b_2_ready' + str(i) + '.csv')

    
    df1 = pd.read_csv(tables_path + "\\B1\\help_tables\\B1b_2_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["MIN"]/10000
    df = df.merge(df1[["COUNTRY", rasters_si[i]]], on = "COUNTRY")
    
df.drop(["OID_", "ZONE_CODE", "MIN"], axis=1, inplace=True)
output_path = tables_path + '\\READY_country_level\\B1b_2_READY.xlsx'
df.to_excel(output_path)

100%|████████████████████████████████████████████████████████████████████████████████| 45/45 [1:03:44<00:00, 84.98s/it]


## B2_1

In [ ]:
rasters_hyde[6]

In [ ]:
env.workspace = hyde_path

# Computing weights

pop_dens = rasters_hyde[6]
sum_of_popdens = ZonalStatistics(regions_shape_path, VAR, pop_dens, "SUM", "DATA")
w_s = Raster(pop_dens) / Raster(sum_of_popdens)

# Adjusting SI rasters with weights 

env.workspace = dtb_path
for i in range(len(rasters_si)):
    out = Raster(rasters_si[i]) * Raster(w_s)
    out.save(tables_path + '\\B2\\B2_1\\rasters_si\\' + rasters_si[i])

In [ ]:
env.workspace = tables_path + '\\B2\\B2_1\\rasters_si\\'

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[0],
                        tables_path +'\\B2\\B2_1\\RESULTS', "DATA", "STD")
tt(tables_path +'\\B2\\B2_1\\RESULTS', tables_path + '\\B2\\help_tables', 'B2_1_ready.csv')

df = pd.read_csv(tables_path + "\\B2\\help_tables\\B2_1_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["STD"]/10000

for i in tqdm(range(1, 46)):
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[i],
                        tables_path +'\\B2\\B2_1\\RESULTS', "DATA", "STD")
    tt(tables_path +'\\B2\\B2_1\\RESULTS', tables_path + '\\B2\\help_tables', 'B2_1_ready' + str(i) + '.csv')
    df1 = pd.read_csv(tables_path + "\\B2\\help_tables\\B2_1_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["STD"]/10000
    df = df.merge(df1[["REGIONID", rasters_si[i]]], on = "REGIONID")
    
df.drop(["OID_", "ZONE_CODE", "STD"], axis=1, inplace=True)
output_path = tables_path + '\\READY\\B2_1_READY.xlsx'
df.to_excel(output_path)

## B2b_1

In [16]:
env.workspace = hyde_path
rasters_hyde[6]

'popd_1500AD.asc.tif'

In [17]:
env.workspace = hyde_path

# Computing weights

pop_dens = rasters_hyde[6]
sum_of_popdens = ZonalStatistics(regions_shape_path, VAR, pop_dens, "SUM", "DATA")
w_s = Raster(pop_dens) / Raster(sum_of_popdens)

# Adjusting SI rasters with weights 

env.workspace = dtb_path
for i in range(len(rasters_si)):
    out = Raster(rasters_si[i]) * Raster(w_s)
    out.save(tables_path + '\\B2\\B2_1\\rasters_si\\' + rasters_si[i])

In [18]:
env.workspace = tables_path + '\\B2\\B2_1\\rasters_si\\'
sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[0], "SUM", "DATA")

env.workspace = dtb_path
si_minus_sum_w_si = Raster(rasters_si[0]) - Raster(sum_w_si)

square = Square(si_minus_sum_w_si)
under_root = Raster(w_s) * Raster(square)
out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                        tables_path +'\\B2\\B2b_1\\RESULTS', "DATA", "MAXIMUM")

tt(tables_path +'\\B2\\B2b_1\\RESULTS', tables_path + '\\B2\\help_tables', 'B2b_1_ready.csv')

df = pd.read_csv(tables_path + "\\B2\\help_tables\\B2b_1_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["MAX"]/10000

In [19]:
for i in tqdm(range(1, 46)):
    env.workspace = tables_path + '\\B2\\B2_1\\rasters_si\\'
    sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[i], "SUM", "DATA")

    env.workspace = dtb_path
    si_minus_sum_w_si = Raster(rasters_si[i]) - Raster(sum_w_si)

    square = Square(si_minus_sum_w_si)
    under_root = Raster(w_s) * Raster(square)
    out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                            tables_path +'\\B2\\B2b_1\\RESULTS', "DATA", "MAXIMUM")

    tt(tables_path +'\\B2\\B2b_1\\RESULTS', tables_path + '\\B2\\help_tables', 'B2b_1_ready' + str(i) + '.csv')

    
    df1 = pd.read_csv(tables_path + "\\B2\\help_tables\\B2b_1_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["MAX"]/10000
    df = df.merge(df1[["COUNTRY", rasters_si[i]]], on = "COUNTRY")
    
df.drop(["OID_", "ZONE_CODE", "MAX"], axis=1, inplace=True)
output_path = tables_path + '\\READY_country_level\\B2b_1_READY.xlsx'
df.to_excel(output_path)

100%|███████████████████████████████████████████████████████████████████████████████| 45/45 [1:31:14<00:00, 121.66s/it]


## B2_2

In [ ]:
env.workspace = hyde_path

# Computing weights

pop_dens = rasters_hyde[6]
sum_of_popdens = ZonalStatistics(regions_shape_path, VAR, pop_dens, "SUM", "DATA")
w_s = Raster(pop_dens) / Raster(sum_of_popdens)

env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))
# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")

# Creating SI rasters which satisfy the constraint and are multiplied by weights:
# 1) max SI across all crops >= 0.1 (suitable cell for agriculture)

for i in tqdm(range(46)):
    out = Raster(rasters_si[i]) * Raster(cutoff_constraint) * Raster(w_s)
    Float(out).save(tables_path + '\\B2\\B2_2\\rasters_si\\' + rasters_si[i])

In [ ]:
env.workspace = tables_path + '\\B2\\B2_2\\rasters_si\\'

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[0],
                        tables_path +'\\B2\\B2_2\\RESULTS', "DATA", "STD")
tt(tables_path +'\\B2\\B2_2\\RESULTS', tables_path + '\\B2\\help_tables', 'B2_2_ready.csv')

df = pd.read_csv(tables_path + "\\B2\\help_tables\\B2_2_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["STD"]/10000

for i in tqdm(range(1, 46)):
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, rasters_si[i],
                        tables_path +'\\B2\\B2_2\\RESULTS', "DATA", "STD")
    tt(tables_path +'\\B2\\B2_2\\RESULTS', tables_path + '\\B2\\help_tables', 'B2_2_ready' + str(i) + '.csv')
    df1 = pd.read_csv(tables_path + "\\B2\\help_tables\\B2_2_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["STD"]/10000
    df = df.merge(df1[["REGIONID", rasters_si[i]]], on = "REGIONID")
    
df.drop(["OID_", "ZONE_CODE", "STD"], axis=1, inplace=True)
output_path = tables_path + '\\READY\\B2_2_READY.xlsx'
df.to_excel(output_path)

## B2b_2

In [20]:
env.workspace = hyde_path

# Computing weights

pop_dens = rasters_hyde[6]
sum_of_popdens = ZonalStatistics(regions_shape_path, VAR, pop_dens, "SUM", "DATA")
w_s = Raster(pop_dens) / Raster(sum_of_popdens)

env.workspace = dtb_path

# Defining maximum SI in each cell
max_si_all_cells = Float(arcpy.sa.CellStatistics(rasters_si, 'MAXIMUM', 'DATA'))
# Avoiding cells where maximum SI <= 0.1
cutoff_constraint = SetNull(max_si_all_cells, 1, "VALUE <= 1000")

# Creating SI rasters which satisfy the constraint and are multiplied by weights:
# 1) max SI across all crops >= 0.1 (suitable cell for agriculture)

for i in tqdm(range(46)):
    out = Raster(rasters_si[i]) * Raster(cutoff_constraint) * Raster(w_s)
    Float(out).save(tables_path + '\\B2\\B2_2\\rasters_si\\' + rasters_si[i])


env.workspace = tables_path + '\\B2\\B2_2\\rasters_si\\'
sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[0], "SUM", "DATA")

env.workspace = dtb_path
si_minus_sum_w_si = Raster(rasters_si[0]) - Raster(sum_w_si)

square = Square(si_minus_sum_w_si)
under_root = Raster(w_s) * Raster(square)
out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                        tables_path +'\\B2\\B2b_2\\RESULTS', "DATA", "MAXIMUM")
tt(tables_path +'\\B2\\B2b_2\\RESULTS', tables_path + '\\B2\\help_tables', 'B2b_2_ready.csv')
df = pd.read_csv(tables_path + "\\B2\\help_tables\\B2b_2_ready.csv", decimal = ",", sep =";")
df[rasters_si[0]] = df["MAX"]/10000


for i in tqdm(range(1, 46)):
    env.workspace = tables_path + '\\B2\\B2_2\\rasters_si\\'
    sum_w_si = ZonalStatistics(regions_shape_path, VAR, rasters_si[i], "SUM", "DATA")

    env.workspace = dtb_path
    si_minus_sum_w_si = Raster(rasters_si[i]) - Raster(sum_w_si)

    square = Square(si_minus_sum_w_si)
    under_root = Raster(w_s) * Raster(square)
    out = ZonalStatistics(regions_shape_path, VAR, under_root, "SUM", "DATA")
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, SquareRoot(out),
                            tables_path +'\\B2\\B2b_2\\RESULTS', "DATA", "MAXIMUM")

    tt(tables_path +'\\B2\\B2b_2\\RESULTS', tables_path + '\\B2\\help_tables', 'B2b_2_ready' + str(i) + '.csv')

    
    df1 = pd.read_csv(tables_path + "\\B2\\help_tables\\B2b_2_ready" + str(i) + ".csv", decimal = ",", sep =";")
    df1[rasters_si[i]] = df1["MAX"]/10000
    df = df.merge(df1[["COUNTRY", rasters_si[i]]], on = "COUNTRY")
    
df.drop(["OID_", "ZONE_CODE", "MAX"], axis=1, inplace=True)
output_path = tables_path + '\\READY_country_level\\B2b_2_READY.xlsx'
df.to_excel(output_path)

100%|████████████████████████████████████████████████████████████████████████████████| 45/45 [1:03:04<00:00, 84.09s/it]


## D1

In [11]:
env.workspace = fao_prod_path

sum_across_crops = Float(arcpy.sa.CellStatistics(rasters_fao_prod, 'SUM', 'DATA'))
denumerator = ZonalStatistics(regions_shape_path, VAR, sum_across_crops, "SUM", "DATA")

for i in tqdm(range(len(rasters_fao_prod))):
    numerator = ZonalStatistics(regions_shape_path, VAR, rasters_fao_prod[i], "SUM", "DATA")
    frac_score = Raster(numerator) / Raster(denumerator)
    sq = Square(frac_score)
    Float(sq).save(tables_path +'\\D1\\frac_squared\\' + rasters_fao_prod[i])
    
    
env.workspace = tables_path + '\\D1\\frac_squared\\'

out = Float(arcpy.sa.CellStatistics(rasters_fao_prod, 'SUM', 'DATA'))
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, 1-Float(out),
                        tables_path +'\\D1\\RESULTS', "DATA", "MAXIMUM")
tt(tables_path +'\\D1\\RESULTS', tables_path + '\\READY_country_level', 'D1_READY.csv')

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [15:14<00:00, 33.86s/it]


<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\D1_READY.csv'>

## E

In [124]:
# Correct order of files and names for E indices

indexE_rasters_hyde = ['popd_1500AD.asc.tif',
 'popd_1700AD.asc.tif', 'cropland1500AD.asc.tif',
 'cropland1700AD.asc.tif', 'tot_rainfed1500AD.asc.tif',
 'tot_rainfed1700AD.asc.tif', 'rf_rice1500AD.asc.tif',
 'rf_rice1700AD.asc.tif', 'rf_norice1500AD.asc.tif',
 'rf_norice1700AD.asc.tif', 'grazing1500AD.asc.tif',
 'grazing1700AD.asc.tif', 'rangeland1500AD.asc.tif',
 'rangeland1700AD.asc.tif', 'pasture1500AD.asc.tif',
 'pasture1700AD.asc.tif']

indexE_names = ["E0a", "E0b", "E1a", "E1b", "E2a", "E2b", "E3a", "E3b", "E4a", "E4b", "E5a", "E5b", "E6a", "E6b", "E7a", "E7b"]

In [126]:
env.workspace = hyde_path

arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, indexE_rasters_hyde[0],
                        tables_path +'\\E\\RESULTS', "DATA", "MEAN")
tt(tables_path +'\\E\\RESULTS', tables_path + '\\E\\help_tables', 'E0_ready.csv')


df = pd.read_csv(tables_path + "\\E\\help_tables\\E0_ready.csv", decimal = ",", sep =";")
df[indexE_names[0]] = df["MEAN"]
for i in tqdm(range(1, 16)):
    arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, indexE_rasters_hyde[i],
                        tables_path +'\\E\\RESULTS', "DATA", "MEAN")
    tt(tables_path +'\\E\\RESULTS', tables_path + '\\E\\help_tables', 'E' + str(i) + '_ready.csv')
    df1 = pd.read_csv(tables_path + '\\E\\help_tables\\E' + str(i) + '_ready.csv', decimal = ",", sep =";")
    df1[indexE_names[i]] = df1["MEAN"]
    df = df.merge(df1[["COUNTRY", indexE_names[i]]], on = "COUNTRY", how='outer')
    
df.drop(["OID_", "ZONE_CODE", "MEAN"], axis=1, inplace=True)

output_path = tables_path + '\\READY_country_level\\E_READY.xlsx'
df.to_excel(output_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [08:03<00:00, 32.25s/it]


## F1a, F1b, F2, F3

In [ ]:
arcpy.Dissolve_management(regions_shape_path, countries_shape_path, "COUNTRY")

In [227]:
env.workspace = trade_path
trade_points = ['trade_1700_full.shp', 'trade_1300.shp', 'ancient_ports.shp', 'romanroads_points.shp']
indexF_names = ["F1a", "F1b", "F2", "F3"]

In [228]:
# Intersecting points and shape files of regions

for i in range(4):
    target_features = countries_shape_path
    join_features = trade_points[i]
    out_feature_class =  tables_path + '\\F\\help_tables\\F' + str(i)

    arcpy.analysis.SpatialJoin(target_features, join_features, out_feature_class)

In [229]:
# Creating a table

fc = tables_path + '\\F\\help_tables\\F0.shp'
fields = ['COUNTRY', 'Join_Count']
res = []
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        res.append(row)
df = pd.DataFrame(res, columns = ['COUNTRY', indexF_names[0]])

for i in range(1, 4): 
    fc = tables_path + '\\F\\help_tables\\F' + str(i) + '.shp'
    fields = ['COUNTRY', 'Join_Count']
    res = []
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        for row in cursor:
            res.append(row)
    df = df.merge(pd.DataFrame(res, columns = ['COUNTRY', indexF_names[i]]), on = 'COUNTRY')

output_path = tables_path + '\\READY_country_level\\F1-F3_READY.xlsx'
df.to_excel(output_path)

## F4a, F4b, F5, F6

In [221]:
env.workspace = trade_path
trade_points = ['trade_1700_full.shp', 'trade_1300.shp', 'ancient_ports.shp', 'romanroads_points.shp']
indexF_names = ["F4a", "F4b", "F5", "F6"]

In [222]:
# Creating Feature Layer with centroids of regions

inFeat = countries_shape_path
outFeat = tables_path + '\\F\\centroids'
arcpy.FeatureToPoint_management(inFeat, outFeat, "CENTROID")

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\F\\centroids.shp'>

In [223]:
in_features = tables_path + '\\F\\centroids.shp'
near_features = trade_points[0]
arcpy.analysis.Near(in_features, near_features, method = 'GEODESIC')

fc = tables_path + '\\F\\centroids.shp'
fields = ['COUNTRY', 'NEAR_DIST']
res = []
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        res.append(row)
df = pd.DataFrame(res, columns = ['COUNTRY', indexF_names[0]])


for i in range(1, 4):
    near_features = trade_points[i]
    arcpy.analysis.Near(in_features, near_features, method = 'GEODESIC')
    
    fc = tables_path + '\\F\\centroids.shp'
    fields = ['COUNTRY', 'NEAR_DIST']
    res = []
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        for row in cursor:
            res.append(row)
    df = df.merge(pd.DataFrame(res, columns = ['COUNTRY', indexF_names[i]]), on = 'COUNTRY')

# Making distance_unit = 'kilometers'    
df[indexF_names] = df[indexF_names]/1000

# Saving
output_path = tables_path + '\\READY_country_level\\F4-F6_READY.xlsx'
df.to_excel(output_path)

## F7a, F7b, F8, F9

In [230]:
env.workspace = trade_path
trade_points = ['trade_1700_full.shp', 'trade_1300.shp', 'ancient_ports.shp', 'romanroads_points.shp']
indexF_names = ["F7a", "F7b", "F8", "F9"]

In [232]:
in_source_data = trade_path + '\\' + trade_points[0]
in_cost_raster = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Human Mobility Index\Human Mobility Index\HMISea.tif"
cost_dist = CostDistance(in_source_data, in_cost_raster)

in_point_features = tables_path + '\\F\\centroids.shp'
in_raster = cost_dist
out_point_features = tables_path + '\\F\\' + indexF_names[0] + '.shp'
ExtractValuesToPoints(in_point_features, in_raster, out_point_features)

fc = out_point_features
fields = ['COUNTRY', 'RASTERVALU']
res = []
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        res.append(row)
df = pd.DataFrame(res, columns = ['COUNTRY', indexF_names[0]])


In [233]:
for i in range(1, 4):
    in_source_data = trade_path + '\\' + trade_points[i]
    in_cost_raster = r"C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Human Mobility Index\Human Mobility Index\HMISea.tif"
    cost_dist = CostDistance(in_source_data, in_cost_raster)
    
    in_point_features = tables_path + '\\F\\centroids.shp'
    in_raster = cost_dist
    out_point_features = tables_path + '\\F\\' + indexF_names[i] + '.shp'
    ExtractValuesToPoints(in_point_features, in_raster, out_point_features)
    
    fc = out_point_features
    fields = ['COUNTRY', 'RASTERVALU']
    res = []
    with arcpy.da.SearchCursor(fc, fields) as cursor:
        for row in cursor:
            res.append(row)
    df = df.merge(pd.DataFrame(res, columns = ['COUNTRY', indexF_names[i]]), on = 'COUNTRY')

# Making distance_unit = 'kilometers'    
df[indexF_names] = df[indexF_names]/1000

# Saving
output_path = tables_path + '\\READY_country_level\\F7-F9_READY.xlsx'
df.to_excel(output_path)

## G1

In [257]:
env.workspace = hmi_path
rasters_hmi = arcpy.ListRasters()

In [258]:
arcpy.Resample_management(rasters_hmi[0], tables_path + "\\hmi_resampled.tif", "1000", "BILINEAR")

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\hmi_resampled.tif'>

In [259]:
def raster_to_numpy(raster):
    descData = arcpy.Describe(raster)
    cellSize = descData.meanCellHeight
    extent = descData.Extent
    spatialReference = descData.spatialReference
    pnt = arcpy.Point(extent.XMin,extent.YMin)
    return arcpy.RasterToNumPyArray(raster, lower_left_corner=pnt, nodata_to_value= -5)

X = raster_to_numpy(tables_path + "\\hmi_resampled.tif")

In [260]:
avg_hmi = np.zeros((X.shape[0], X.shape[1]))

for i in tqdm(range(1, X.shape[0]-1)):
    for j in range(1, X.shape[1]-1):
        if X[i][j] != -5:
            n = 0
            S = 0
            if X[i-1][j-1] != -5:
                n+=1
                S += 1.41421 * (X[i-1][j-1] + X[i][j])/2
            elif X[i-1][j+1] != -5:
                n+=1
                S += 1.41421 * (X[i-1][j+1] + X[i][j])/2
            elif X[i+1][j-1] != -5:
                n+=1
                S += 1.41421 * (X[i+1][j-1] + X[i][j])/2
            elif X[i-1][j+1] != -5:
                n+=1
                S += 1.41421 * (X[i-1][j+1] + X[i][j])/2
            elif X[i-1][j] != -5:
                n+=1
                S += 1 * (X[i-1][j] + X[i][j])/2
            elif X[i+1][j] != -5:
                n+=1
                S += 1 * (X[i+1][j] + X[i][j])/2
            elif X[i][j-1] != -5:
                n+=1
                S += 1 * (X[i][j-1] + X[i][j])/2
            elif X[i][j+1] != -5:
                n+=1
                S += 1 * (X[i][j+1] + X[i][j])/2

            if n>0:
                avg_hmi[i][j] = S/n
                
avg_hmi = avg_hmi.astype("float32")

100%|████████████████████████████████████████████████████████████████████████████| 11867/11867 [44:08<00:00,  4.48it/s]


In [264]:
avg_hmi.shape

(11869, 40075)

In [262]:
def numpy_to_raster(numpy_array):
    descData = arcpy.Describe(tables_path + "\\hmi_resampled.tif")
    cellSize = descData.meanCellHeight
    extent = descData.Extent
    spatialReference = descData.spatialReference
    pnt = arcpy.Point(extent.XMin,extent.YMin)
    newRaster = arcpy.NumPyArrayToRaster(numpy_array,pnt,cellSize,cellSize, value_to_nodata = 0)
    arcpy.DefineProjection_management(newRaster,spatialReference)
    return newRaster

out = numpy_to_raster(tables_path + "\\avg_hmi.tif")
#out.save(tables_path + "\\avg_hmi.tif")
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, out,
                        tables_path +'\\G1\\RESULTS', "DATA", "MEAN")
tt(tables_path +'\\G1\\RESULTS', tables_path + '\\READY_country_level', 'G1_READY.csv')

ValueError: Argument in_array: A two or three dimensional NumPy array is required.

## G1_b

In [234]:
with open('kernel_G.txt', 'w') as f:
    f.write('3 3\n0.653762 0.46228 0.653762\n0.46228 4.46417 0.46228\n0.653762 0.46228 0.653762')

In [235]:
env.workspace = hmi_path
rasters_hmi = arcpy.ListRasters()
OutRas = FocalStatistics(rasters_hmi[0], NbrWeight('kernel_G.txt'), "SUM", "NODATA")
out = Raster(OutRas) / 8

In [236]:
arcpy.gp.ZonalStatisticsAsTable_sa(countries_shape_path, VAR, out,
                        tables_path +'\\G1\\RESULTS', "DATA", "MEAN")
tt(tables_path +'\\G1\\RESULTS', tables_path + '\\READY_country_level', 'G1_b_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\G1_b_READY.csv'>

## G2

In [5]:
env.workspace = hmi_path
rasters_hmi = arcpy.ListRasters()
arcpy.Resample_management(rasters_hmi[1], tables_path + "\\hmisea_resampled.tif", "1000", "BILINEAR")

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\hmisea_resampled.tif'>

In [6]:
def raster_to_numpy(raster):
    descData = arcpy.Describe(raster)
    cellSize = descData.meanCellHeight
    extent = descData.Extent
    spatialReference = descData.spatialReference
    pnt = arcpy.Point(extent.XMin,extent.YMin)
    return arcpy.RasterToNumPyArray(raster, lower_left_corner=pnt, nodata_to_value= -5)

X = raster_to_numpy(tables_path + "\\hmisea_resampled.tif")

In [7]:
avg_hmi = np.zeros((X.shape[0], X.shape[1]))

for i in tqdm(range(1, X.shape[0]-1)):
    for j in range(1, X.shape[1]-1):
        if X[i][j] != -5:
            n = 0
            S = 0
            if X[i-1][j-1] != -5:
                n+=1
                S += 1.41421 * (X[i-1][j-1] + X[i][j])/2
            elif X[i-1][j+1] != -5:
                n+=1
                S += 1.41421 * (X[i-1][j+1] + X[i][j])/2
            elif X[i+1][j-1] != -5:
                n+=1
                S += 1.41421 * (X[i+1][j-1] + X[i][j])/2
            elif X[i-1][j+1] != -5:
                n+=1
                S += 1.41421 * (X[i-1][j+1] + X[i][j])/2
            elif X[i-1][j] != -5:
                n+=1
                S += 1 * (X[i-1][j] + X[i][j])/2
            elif X[i+1][j] != -5:
                n+=1
                S += 1 * (X[i+1][j] + X[i][j])/2
            elif X[i][j-1] != -5:
                n+=1
                S += 1 * (X[i][j-1] + X[i][j])/2
            elif X[i][j+1] != -5:
                n+=1
                S += 1 * (X[i][j+1] + X[i][j])/2

            if n>0:
                avg_hmi[i][j] = S/n
                
avg_hmi = avg_hmi.astype("float32")

100%|████████████████████████████████████████████████████████████████████████████| 11867/11867 [38:22<00:00,  5.15it/s]


In [ ]:
def numpy_to_raster(numpy_array):
    descData = arcpy.Describe(tables_path + "\\hmisea_resampled.tif")
    cellSize = descData.meanCellHeight
    extent = descData.Extent
    spatialReference = descData.spatialReference
    pnt = arcpy.Point(extent.XMin,extent.YMin)
    newRaster = arcpy.NumPyArrayToRaster(numpy_array,pnt,cellSize,cellSize, value_to_nodata = 0)
    arcpy.DefineProjection_management(newRaster,spatialReference)
    return newRaster

out = numpy_to_raster(avg_hmi)
arcpy.gp.ZonalStatisticsAsTable_sa(regions_shape_path, VAR, out,
                        tables_path +'\\G2\\RESULTS', "DATA", "MEAN")
tt(tables_path +'\\G2\\RESULTS', tables_path + '\\READY', 'G2_READY.csv')

## G2_b

In [237]:
OutRas = FocalStatistics(rasters_hmi[1], NbrWeight('kernel_G.txt'), "SUM", "NODATA")
out = Raster(OutRas) / 8

In [238]:
arcpy.gp.ZonalStatisticsAsTable_sa(countries_shape_path, VAR, out,
                        tables_path +'\\G2\\RESULTS', "DATA", "MEAN")
tt(tables_path +'\\G2\\RESULTS', tables_path + '\\READY_country_level', 'G2_b_READY.csv')

<Result 'C:\\Users\\Nikita Gordonov.LAPTOP-72M1R5VQ\\OneDrive\\Рабочий стол\\RA\\Nikita indices\\READY_country_level\\G2_b_READY.csv'>

## Share of cells that are suitable for agriculture

In [265]:
df = pd.read_excel(r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Nikita indices\READY_country_level\B0_1_READY.xlsx')
df1 = pd.read_excel(r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Nikita indices\READY_country_level\B0_2_READY.xlsx')

In [266]:
data = df.merge(df1, how= 'left', on ='COUNTRY')
data = data[['COUNTRY','COUNT_x', 'COUNT_y']]
data.fillna(0, inplace=True)
data['share'] = data['COUNT_y']/data['COUNT_x']
data

,COUNTRY,COUNT_x,COUNT_y,share
0,Azerbaijan,1320,1261.0,0.955303
1,Hong Kong,13,13.0,1.000000
2,Nigeria,10781,10201.0,0.946202
3,Puerto Rico,109,109.0,1.000000
4,Trinidad and Tobago,64,64.0,1.000000
...,...,...,...,...
214,British Virgin Islands,2,1.0,0.500000
215,"Virgin Islands, U.S.",3,3.0,1.000000
216,Vanuatu,144,127.0,0.881944
217,Wallis and Futuna,2,0.0,0.000000


In [267]:
data[['COUNTRY', 'share']].to_excel(r'C:\Users\Nikita Gordonov.LAPTOP-72M1R5VQ\OneDrive\Рабочий стол\RA\Nikita indices\READY_country_level\share_of_cells_suitable_for_agriculture.xlsx')

In [239]:
indices = ['A1_1', 'A1_2', 'A1_3', 'A1_4', 
           'A2a_1', 'A2a_2', 'A2a_3', 'A2a_4', 
           'A2b_1', 'A2b_2', 'A2b_3', 'A2b_4',
           'A3_1', 'A3_2', 'A3_3', 'A3_4', 
           'A4_1', 'A4_2', 'A4_3', 'A4_4', 
           'A5_1', 'A5_2', 'A5_3', 'A5_4', 
           'D1']
df = pd.read_csv(tables_path + '\\READY_country_level\\' + indices[0] + '_READY.csv', sep=';', decimal=',')
df.rename(columns ={'MIN': 'A1_1'}, inplace=True)


In [240]:
for i in range(1, len(indices)):
    try: 
        df1 = pd.read_csv(tables_path + '\\READY_country_level\\' + indices[i] + '_READY.csv', sep=';', decimal=',')
        df1.rename(columns ={'MIN': indices[i], "MAX": indices[i]}, inplace=True)
        df = df.merge(df1[['COUNTRY', indices[i]]], on='COUNTRY', how='left')
    except: 
        df1 = pd.read_excel(tables_path + '\\READY_country_level\\' + indices[i] + '_READY.xlsx')
        df = df.merge(df1[['COUNTRY', indices[i]]], on='COUNTRY', how='left')

In [241]:
#df.to_excel(tables_path + '\\READY_country_level\\A1_1-D1.xlsx')

In [268]:
df = pd.read_excel(tables_path + '\\READY_country_level\\A1_1-D1.xlsx')
df1 = pd.read_excel(tables_path + '\\READY_country_level\\E_READY.xlsx')
df = df.merge(df1[['COUNTRY', 'E0a', 'E0b', 'E1a', 'E1b',
       'E2a', 'E2b', 'E3a', 'E3b', 'E4a', 'E4b', 'E5a', 'E5b', 'E6a', 'E6b',
       'E7a', 'E7b']], on='COUNTRY', how ='left')
df1 = pd.read_excel(tables_path + '\\READY_country_level\\F1-F3_READY.xlsx')
df2 = pd.read_excel(tables_path + '\\READY_country_level\\F4-F6_READY.xlsx')
df1 = df1.merge(df2[['COUNTRY', 'F4a', 'F4b', 'F5', 'F6']], on='COUNTRY', how ='left')
df3 = pd.read_excel(tables_path + '\\READY_country_level\\F7-F9_READY.xlsx')
df1 = df1.merge(df3[['COUNTRY', 'F7a', 'F7b', 'F8', 'F9']], on='COUNTRY', how ='left')
df1.loc[df1["COUNTRY"] == "Côte d'Ivoire", "COUNTRY"] = "C_te d'Ivoire"
df1.loc[df1["COUNTRY"] == "São Tomé and Príncipe", "COUNTRY"] = "S_o Tom_ and Pr_ncipe"

df = df.merge(df1, on='COUNTRY', how ='left')

In [270]:
df1 = pd.read_csv(tables_path + '\\READY_country_level\\G1_b_READY.csv', sep=';', decimal=',')
df = df.merge(df1[['COUNTRY', 'G1_b']], on='COUNTRY', how ='left')
df1 = pd.read_csv(tables_path + '\\READY_country_level\\G2_b_READY.csv', sep=';', decimal=',')
df = df.merge(df1[['COUNTRY', 'G2_b']], on='COUNTRY', how ='left')

In [271]:
df.to_excel(tables_path + '\\READY_country_level\\country-level_indices.xlsx')